In [3]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine

# folium 한글 설치
!pip install git+https://github.com/python-visualization/branca.git@master

# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 6.5MB 16.5MB/s 
     |████████████████████████████████| 14.8MB 294kB/s 
  Created wheel for utm: filename=utm-0.6.0-cp36-none-any.whl size=6095 sha256=b60776eccf88f35215790e1877d163469d6d75c2e86ab85da12ecd0284bac937
  Stored in directory: /root/.cache/pip/wheels/9a/76/b2/58ad7ade8b0475c577f80ee8023d7164fd561bb45c20b4beab
Successfully built utm
  Cloning https://github.com/python-visualization/branca.git (to revision master) to /tmp/pip-req-build-ch85tfbc
  Running command git clone -q https://github.com/python-visualization/branca.git /tmp/pip-req-build-ch85tfbc
  Created wheel for branca: filename=branca-0.4.1+4.gac45f1e-cp36-none-any.whl size=24517 sha256=cb0c36cdfacb207b70a4306bc67559b1ca810a19e86f081ae1bebe137c9561ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-47w6vwwe/wheels/14/bc/41/d6078abe73beea93d98c014bba54c395224cabb54f95b9d248
Successfully built branca
ERROR: datascience 0.1

In [4]:
# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 144628 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [5]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [6]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *

In [5]:
# 현재 각 학교주변 있는 좌표들 구하기 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 
from tqdm.notebook import tqdm

def point_school(df_pop, df_school):
  lscp = []

  for k in tqdm(range(len(df_pop))): 
    gid_nm = df_pop['gid'].iloc[k]
    x = (df_pop['위도'].iloc[k], df_pop['경도'].iloc[k])
    for i in range(len(df_school)): 
      y = (df_school['위도'].iloc[i], df_school['경도'].iloc[i])
      h = haversine(x,y, unit='m') 
      if h <= 881:  #전체 좌표 중에서 초등학교 881m 있는 것만 필터링
        lscp.append([gid_nm, df_school['학교명'].iloc[i], h])
  
  return lscp 

#### 성북구

In [7]:
# 초등학생 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_초등학생/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_초등학생/성북구_격자250_초등학생.geojson', driver='GeoJSON', encoding = 'utf-8')

In [8]:
#geojson 읽기 
element_pop_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_초등학생/성북구_격자250_초등학생.geojson', encoding = 'utf-8')
element_pop_1

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
468,다사62aa56ab,None,NaN,"POLYGON ((962000.000 1956250.000, 962000.000 1..."
469,다사62aa56ba,None,NaN,"POLYGON ((962000.000 1956500.000, 962000.000 1..."
470,다사62aa56bb,33.00,33.0,"POLYGON ((962000.000 1956750.000, 962000.000 1..."
471,다사62aa57aa,60.00,60.0,"POLYGON ((962000.000 1957000.000, 962000.000 1..."


In [9]:
# 30대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_30대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_30대/성북구_30대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [10]:
#geojson 읽기 
pop_30_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_30대/성북구_30대_격자250.geojson', encoding = 'utf-8')
pop_30_1

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
468,다사62aa56ab,None,NaN,"POLYGON ((962000.000 1956250.000, 962000.000 1..."
469,다사62aa56ba,None,NaN,"POLYGON ((962000.000 1956500.000, 962000.000 1..."
470,다사62aa56bb,92.00,92.0,"POLYGON ((962000.000 1956750.000, 962000.000 1..."
471,다사62aa57aa,186.00,186.0,"POLYGON ((962000.000 1957000.000, 962000.000 1..."


In [11]:
# 40대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_40대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_40대/성북구_40대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [12]:
#geojson 읽기 
pop_40_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_40대/성북구_40대_격자250.geojson', encoding = 'utf-8')
pop_40_1

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
468,다사62aa56ab,None,NaN,"POLYGON ((962000.000 1956250.000, 962000.000 1..."
469,다사62aa56ba,None,NaN,"POLYGON ((962000.000 1956500.000, 962000.000 1..."
470,다사62aa56bb,84.00,84.0,"POLYGON ((962000.000 1956750.000, 962000.000 1..."
471,다사62aa57aa,199.00,199.0,"POLYGON ((962000.000 1957000.000, 962000.000 1..."


In [13]:
# 생산가능인구 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_생산가능인구/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_생산가능인구/성북구_생산가능인구_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [14]:
#geojson 읽기 
pop_all_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_생산가능인구/성북구_생산가능인구_격자250.geojson', encoding = 'utf-8')
pop_all_1

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
468,다사62aa56ab,None,NaN,"POLYGON ((962000.000 1956250.000, 962000.000 1..."
469,다사62aa56ba,None,NaN,"POLYGON ((962000.000 1956500.000, 962000.000 1..."
470,다사62aa56bb,376.00,376.0,"POLYGON ((962000.000 1956750.000, 962000.000 1..."
471,다사62aa57aa,824.00,824.0,"POLYGON ((962000.000 1957000.000, 962000.000 1..."


### 양천구

In [26]:
# 초등학생 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_초등학생/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_초등학생/양천구_격자250_초등학생.geojson', driver='GeoJSON', encoding = 'utf-8')

In [15]:
#geojson 읽기 
element_pop_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_초등학생/양천구_격자250_초등학생.geojson', encoding = 'utf-8')
element_pop_2

,gid,lbl,val,geometry
0,다사40aa45bb,None,NaN,"POLYGON ((940000.000 1945750.000, 940000.000 1..."
1,다사40aa46ab,None,NaN,"POLYGON ((940000.000 1946250.000, 940000.000 1..."
2,다사40aa46ba,None,NaN,"POLYGON ((940000.000 1946500.000, 940000.000 1..."
3,다사40aa48ab,None,NaN,"POLYGON ((940000.000 1948250.000, 940000.000 1..."
4,다사40aa48ba,8.00,8.0,"POLYGON ((940000.000 1948500.000, 940000.000 1..."
...,...,...,...,...
341,다사45bb48bb,None,NaN,"POLYGON ((945750.000 1948750.000, 945750.000 1..."
342,다사45bb49aa,None,NaN,"POLYGON ((945750.000 1949000.000, 945750.000 1..."
343,다사46aa48aa,None,NaN,"POLYGON ((946000.000 1948000.000, 946000.000 1..."
344,다사46aa48ab,None,NaN,"POLYGON ((946000.000 1948250.000, 946000.000 1..."


In [28]:
# 30대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_30대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_30대/양천구_30대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [16]:
#geojson 읽기 
pop_30_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_30대/양천구_30대_격자250.geojson', encoding = 'utf-8')
pop_30_2

,gid,lbl,val,geometry
0,다사40aa45bb,None,NaN,"POLYGON ((940000.000 1945750.000, 940000.000 1..."
1,다사40aa46ab,None,NaN,"POLYGON ((940000.000 1946250.000, 940000.000 1..."
2,다사40aa46ba,None,NaN,"POLYGON ((940000.000 1946500.000, 940000.000 1..."
3,다사40aa48ab,None,NaN,"POLYGON ((940000.000 1948250.000, 940000.000 1..."
4,다사40aa48ba,30.00,30.0,"POLYGON ((940000.000 1948500.000, 940000.000 1..."
...,...,...,...,...
341,다사45bb48bb,None,NaN,"POLYGON ((945750.000 1948750.000, 945750.000 1..."
342,다사45bb49aa,None,NaN,"POLYGON ((945750.000 1949000.000, 945750.000 1..."
343,다사46aa48aa,None,NaN,"POLYGON ((946000.000 1948000.000, 946000.000 1..."
344,다사46aa48ab,None,NaN,"POLYGON ((946000.000 1948250.000, 946000.000 1..."


In [30]:
# 40대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_40대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_40대/양천구_40대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [17]:
#geojson 읽기 
pop_40_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_40대/양천구_40대_격자250.geojson', encoding = 'utf-8')
pop_40_2

,gid,lbl,val,geometry
0,다사40aa45bb,None,NaN,"POLYGON ((940000.000 1945750.000, 940000.000 1..."
1,다사40aa46ab,None,NaN,"POLYGON ((940000.000 1946250.000, 940000.000 1..."
2,다사40aa46ba,None,NaN,"POLYGON ((940000.000 1946500.000, 940000.000 1..."
3,다사40aa48ab,None,NaN,"POLYGON ((940000.000 1948250.000, 940000.000 1..."
4,다사40aa48ba,21.00,21.0,"POLYGON ((940000.000 1948500.000, 940000.000 1..."
...,...,...,...,...
341,다사45bb48bb,None,NaN,"POLYGON ((945750.000 1948750.000, 945750.000 1..."
342,다사45bb49aa,None,NaN,"POLYGON ((945750.000 1949000.000, 945750.000 1..."
343,다사46aa48aa,None,NaN,"POLYGON ((946000.000 1948000.000, 946000.000 1..."
344,다사46aa48ab,None,NaN,"POLYGON ((946000.000 1948250.000, 946000.000 1..."


In [32]:
# 생산가능 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_생산가능인구/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_생산가능인구/양천구_생산가능인구_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [18]:
#geojson 읽기 
pop_all_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_생산가능인구/양천구_생산가능인구_격자250.geojson', encoding = 'utf-8')
pop_all_2

,gid,lbl,val,geometry
0,다사40aa45bb,None,NaN,"POLYGON ((940000.000 1945750.000, 940000.000 1..."
1,다사40aa46ab,None,NaN,"POLYGON ((940000.000 1946250.000, 940000.000 1..."
2,다사40aa46ba,None,NaN,"POLYGON ((940000.000 1946500.000, 940000.000 1..."
3,다사40aa48ab,None,NaN,"POLYGON ((940000.000 1948250.000, 940000.000 1..."
4,다사40aa48ba,145.00,145.0,"POLYGON ((940000.000 1948500.000, 940000.000 1..."
...,...,...,...,...
341,다사45bb48bb,None,NaN,"POLYGON ((945750.000 1948750.000, 945750.000 1..."
342,다사45bb49aa,None,NaN,"POLYGON ((945750.000 1949000.000, 945750.000 1..."
343,다사46aa48aa,None,NaN,"POLYGON ((946000.000 1948000.000, 946000.000 1..."
344,다사46aa48ab,None,NaN,"POLYGON ((946000.000 1948250.000, 946000.000 1..."


### 강북구

In [20]:
# 초등학생 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_초등학생/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_초등학생/강북구_격자250_초등학생.geojson', driver='GeoJSON', encoding = 'utf-8')

In [21]:
#geojson 읽기 
element_pop_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_초등학생/강북구_격자250_초등학생.geojson', encoding = 'utf-8')
element_pop_3

,gid,lbl,val,geometry
0,다사54aa61ba,None,NaN,"POLYGON ((954000.000 1961500.000, 954000.000 1..."
1,다사54aa61bb,None,NaN,"POLYGON ((954000.000 1961750.000, 954000.000 1..."
2,다사54aa62aa,None,NaN,"POLYGON ((954000.000 1962000.000, 954000.000 1..."
3,다사54ab59bb,None,NaN,"POLYGON ((954250.000 1959750.000, 954250.000 1..."
4,다사54ab60ab,None,NaN,"POLYGON ((954250.000 1960250.000, 954250.000 1..."
...,...,...,...,...
437,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
438,다사60aa58ab,69.00,69.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
439,다사60aa58ba,55.00,55.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
440,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [22]:
# 30대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_30대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_30대/강북구_30대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [23]:
#geojson 읽기 
pop_30_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_30대/강북구_30대_격자250.geojson', encoding = 'utf-8')
pop_30_3

,gid,lbl,val,geometry
0,다사54aa61ba,None,NaN,"POLYGON ((954000.000 1961500.000, 954000.000 1..."
1,다사54aa61bb,None,NaN,"POLYGON ((954000.000 1961750.000, 954000.000 1..."
2,다사54aa62aa,None,NaN,"POLYGON ((954000.000 1962000.000, 954000.000 1..."
3,다사54ab59bb,None,NaN,"POLYGON ((954250.000 1959750.000, 954250.000 1..."
4,다사54ab60ab,None,NaN,"POLYGON ((954250.000 1960250.000, 954250.000 1..."
...,...,...,...,...
437,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
438,다사60aa58ab,182.00,182.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
439,다사60aa58ba,71.00,71.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
440,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [24]:
# 40대  
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_40대/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_40대/강북구_40대_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [25]:
#geojson 읽기 
pop_40_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_40대/강북구_40대_격자250.geojson', encoding = 'utf-8')
pop_40_3

,gid,lbl,val,geometry
0,다사54aa61ba,None,NaN,"POLYGON ((954000.000 1961500.000, 954000.000 1..."
1,다사54aa61bb,None,NaN,"POLYGON ((954000.000 1961750.000, 954000.000 1..."
2,다사54aa62aa,None,NaN,"POLYGON ((954000.000 1962000.000, 954000.000 1..."
3,다사54ab59bb,None,NaN,"POLYGON ((954250.000 1959750.000, 954250.000 1..."
4,다사54ab60ab,None,NaN,"POLYGON ((954250.000 1960250.000, 954250.000 1..."
...,...,...,...,...
437,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
438,다사60aa58ab,271.00,271.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
439,다사60aa58ba,122.00,122.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
440,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [26]:
# 생산가능 
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_생산가능인구/vl_blk.shp', encoding = 'cp949')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_생산가능인구/강북구_생산가능인구_격자250.geojson', driver='GeoJSON', encoding = 'utf-8')

In [27]:
#geojson 읽기 
pop_all_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_생산가능인구/강북구_생산가능인구_격자250.geojson', encoding = 'utf-8')
pop_all_3

,gid,lbl,val,geometry
0,다사54aa61ba,None,NaN,"POLYGON ((954000.000 1961500.000, 954000.000 1..."
1,다사54aa61bb,None,NaN,"POLYGON ((954000.000 1961750.000, 954000.000 1..."
2,다사54aa62aa,None,NaN,"POLYGON ((954000.000 1962000.000, 954000.000 1..."
3,다사54ab59bb,None,NaN,"POLYGON ((954250.000 1959750.000, 954250.000 1..."
4,다사54ab60ab,None,NaN,"POLYGON ((954250.000 1960250.000, 954250.000 1..."
...,...,...,...,...
437,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
438,다사60aa58ab,1001.00,1001.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
439,다사60aa58ba,558.00,558.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
440,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


## 동 3개 merge

### 초등인구

In [28]:
element_pop_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_초등학생/성북구_격자250_초등학생.geojson', encoding = 'utf-8')
element_pop_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_초등학생/양천구_격자250_초등학생.geojson', encoding = 'utf-8')
element_pop_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_초등학생/강북구_격자250_초등학생.geojson', encoding = 'utf-8')

In [29]:
element_pop = pd.concat([element_pop_1, element_pop_2, element_pop_3], axis=0).reset_index().drop('index', axis=1)
element_pop

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
1256,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
1257,다사60aa58ab,69.00,69.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
1258,다사60aa58ba,55.00,55.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
1259,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [30]:
# concat 확인
len(element_pop.drop_duplicates())
element_pop = element_pop.drop_duplicates(['gid'])
len(element_pop)

1216

#### 30대

In [31]:
pop_30_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_30대/성북구_30대_격자250.geojson', encoding = 'utf-8')
pop_30_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_30대/양천구_30대_격자250.geojson', encoding = 'utf-8')
pop_30_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_30대/강북구_30대_격자250.geojson', encoding = 'utf-8')

In [32]:
pop_30 = pd.concat([pop_30_1, pop_30_2, pop_30_3], axis=0).reset_index().drop('index', axis=1)
pop_30

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
1256,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
1257,다사60aa58ab,182.00,182.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
1258,다사60aa58ba,71.00,71.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
1259,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [33]:
# concat 확인
len(pop_30.drop_duplicates())
pop_30 = pop_30.drop_duplicates(['gid'])
len(pop_30)

1216

### 40대

In [34]:
pop_40_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_40대/성북구_40대_격자250.geojson', encoding = 'utf-8')
pop_40_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_40대/양천구_40대_격자250.geojson', encoding = 'utf-8')
pop_40_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_40대/강북구_40대_격자250.geojson', encoding = 'utf-8')

In [35]:
pop_40 = pd.concat([pop_40_1, pop_40_2, pop_40_3], axis=0).reset_index().drop('index', axis=1)
pop_40

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
1256,다사60aa58aa,None,NaN,"POLYGON ((960000.000 1958000.000, 960000.000 1..."
1257,다사60aa58ab,271.00,271.0,"POLYGON ((960000.000 1958250.000, 960000.000 1..."
1258,다사60aa58ba,122.00,122.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."
1259,다사60aa58bb,None,NaN,"POLYGON ((960000.000 1958750.000, 960000.000 1..."


In [36]:
# concat 확인
len(pop_40.drop_duplicates())
pop_40 = pop_40.drop_duplicates(['gid'])
len(pop_40)

1216

### 생산가능인구

In [37]:
pop_all_1 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/성북구/성북구_격자250_생산가능인구/성북구_생산가능인구_격자250.geojson', encoding = 'utf-8')
pop_all_2 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/양천구/양천구_격자250_생산가능인구/양천구_생산가능인구_격자250.geojson', encoding = 'utf-8')
pop_all_3 = gpd.read_file('/content/drive/My Drive/Project/Seoul/인구/강북구/강북구_격자250_생산가능인구/강북구_생산가능인구_격자250.geojson', encoding = 'utf-8')

In [38]:
pop_all = pd.concat([pop_all_1, pop_all_2, pop_all_3], axis=0).reset_index().drop('index', axis=1)
pop_all = pop_all.drop_duplicates(['gid'])
pop_all

,gid,lbl,val,geometry
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1..."
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1..."
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1..."
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1..."
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1..."
...,...,...,...,...
1253,다사59bb58ab,1235.00,1235.0,"POLYGON ((959750.000 1958250.000, 959750.000 1..."
1254,다사59bb58ba,1735.00,1735.0,"POLYGON ((959750.000 1958500.000, 959750.000 1..."
1255,다사59bb58bb,259.00,259.0,"POLYGON ((959750.000 1958750.000, 959750.000 1..."
1258,다사60aa58ba,558.00,558.0,"POLYGON ((960000.000 1958500.000, 960000.000 1..."


In [39]:
# concat 확인
len(pop_all.drop_duplicates())
pop_all = pop_all.drop_duplicates(['gid'])
len(pop_all)

1216

## 초등학생 _격자

### utm-k(grs80) -> wgs84

In [40]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

# Projection 정의
# UTM-K
proj_UTMK = Proj(init='epsg:5179') #GRS80 UTMK

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

# centroid 찍고 X, Y 반환
element_pop['utm_Y'] = element_pop['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[1]) #geometry에서 utm_Y 반환 
element_pop['utm_Y'] = element_pop['utm_Y'].map(mapper)

element_pop['utm_X'] = element_pop['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[0]) #geometry에서 utm_X 반환 
element_pop['utm_X'] = element_pop['utm_X'].map(mapper)

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['utm_X'], df['utm_Y']), index=['utm_X', 'utm_Y'])


element_pop[['경도', '위도']] = element_pop.apply(transform_utmk_to_w84, axis=1)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [41]:
element_pop

,gid,lbl,val,geometry,utm_Y,utm_X,경도,위도
0,다사53ba58bb,None,NaN,"POLYGON ((953500.000 1958750.000, 953500.000 1...",1958875.0,953625.0,126.974439,37.628162
1,다사53ba59aa,None,NaN,"POLYGON ((953500.000 1959000.000, 953500.000 1...",1959125.0,953625.0,126.974423,37.630416
2,다사53ba59ab,None,NaN,"POLYGON ((953500.000 1959250.000, 953500.000 1...",1959375.0,953625.0,126.974408,37.632669
3,다사53bb55ab,None,NaN,"POLYGON ((953750.000 1955250.000, 953750.000 1...",1955375.0,953875.0,126.977493,37.596629
4,다사53bb55ba,None,NaN,"POLYGON ((953750.000 1955500.000, 953750.000 1...",1955625.0,953875.0,126.977477,37.598882
...,...,...,...,...,...,...,...,...
1253,다사59bb58ab,116.00,116.0,"POLYGON ((959750.000 1958250.000, 959750.000 1...",1958375.0,959875.0,127.045295,37.623950
1254,다사59bb58ba,58.00,58.0,"POLYGON ((959750.000 1958500.000, 959750.000 1...",1958625.0,959875.0,127.045281,37.626203
1255,다사59bb58bb,7.00,7.0,"POLYGON ((959750.000 1958750.000, 959750.000 1...",1958875.0,959875.0,127.045267,37.628457
1258,다사60aa58ba,55.00,55.0,"POLYGON ((960000.000 1958500.000, 960000.000 1...",1958625.0,960125.0,127.048114,37.626214


In [42]:
element_pop = element_pop.drop(['lbl', 'utm_Y', 'utm_X', 'geometry'], axis=1)
element_pop['val'] = element_pop['val'].fillna(0)
element_pop

,gid,val,경도,위도
0,다사53ba58bb,0.0,126.974439,37.628162
1,다사53ba59aa,0.0,126.974423,37.630416
2,다사53ba59ab,0.0,126.974408,37.632669
3,다사53bb55ab,0.0,126.977493,37.596629
4,다사53bb55ba,0.0,126.977477,37.598882
...,...,...,...,...
1253,다사59bb58ab,116.0,127.045295,37.623950
1254,다사59bb58ba,58.0,127.045281,37.626203
1255,다사59bb58bb,7.0,127.045267,37.628457
1258,다사60aa58ba,55.0,127.048114,37.626214


### 행정동 mapping

In [43]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
419,1174069000,강동구,둔촌제1동,37.523145,127.140435,"MULTIPOLYGON (((127.14580 37.52199, 127.14564 ..."
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."


In [44]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(element_pop['경도'], element_pop['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
element_pop = gpd.GeoDataFrame(element_pop, crs=crs, geometry=geometry)
element_pop

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,gid,val,경도,위도,geometry
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816)
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042)
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267)
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663)
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888)
...,...,...,...,...,...
1253,다사59bb58ab,116.0,127.045295,37.623950,POINT (127.04529 37.62395)
1254,다사59bb58ba,58.0,127.045281,37.626203,POINT (127.04528 37.62620)
1255,다사59bb58bb,7.0,127.045267,37.628457,POINT (127.04527 37.62846)
1258,다사60aa58ba,55.0,127.048114,37.626214,POINT (127.04811 37.62621)


In [45]:
# 행정동 포함여부 확인 
element_pop['행정동코드'] = np.zeros(len(element_pop))
element_pop['시군구명'] = np.zeros(len(element_pop))
element_pop['행정동'] = np.zeros(len(element_pop))

for i in tqdm(element_pop.index):
    for j in dong_geo.index:
        if element_pop.geometry[i].within(dong_geo.geometry[j]):
            element_pop['행정동'][i] = dong_geo.읍면동명[j]
            element_pop['행정동코드'][i] = dong_geo.행정동코드[j]
            element_pop['시군구명'][i] = dong_geo.시군구명[j]
            
element_pop['행정동코드'] = element_pop['행정동코드'].astype(int)
element_pop

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path whi

,gid,val,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816),1111056000,종로구,평창동
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042),1111056000,종로구,평창동
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267),0,0,0
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663),1111056000,종로구,평창동
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888),1111056000,종로구,평창동
...,...,...,...,...,...,...,...,...
1253,다사59bb58ab,116.0,127.045295,37.623950,POINT (127.04529 37.62395),1130560800,강북구,번3동
1254,다사59bb58ba,58.0,127.045281,37.626203,POINT (127.04528 37.62620),1130560800,강북구,번3동
1255,다사59bb58bb,7.0,127.045267,37.628457,POINT (127.04527 37.62846),1130560800,강북구,번3동
1258,다사60aa58ba,55.0,127.048114,37.626214,POINT (127.04811 37.62621),1130560800,강북구,번3동


In [50]:
print(len(element_pop[element_pop['행정동코드'] == 1130560300]) + len(element_pop[element_pop['행정동코드'] == 1147055000]) + len(element_pop[element_pop['행정동코드'] == 1129066000])) 
len(element_pop[element_pop['행정동코드'] == 1130560300]) , len(element_pop[element_pop['행정동코드'] == 1147055000]) , len(element_pop[element_pop['행정동코드'] == 1129066000])

62


(16, 33, 13)

In [51]:
df_element = element_pop.copy()
mask1 = df_element['행정동']==0 
mask2 = df_element['행정동코드'] == 1129066000 #'길음1동'
mask3 = df_element['행정동코드'] == 1147055000 #'목5동'
mask4 = df_element['행정동코드'] == 1130560300 #'번2동'

df_element = df_element[~mask1]
df_element = df_element[mask2 | mask3 | mask4].reset_index().drop(['index'], axis=1)
df_element.columns = ['gid', '초등학생_인구', '경도', '위도', 'geometry', '행정동코드','시군구명','행정동']
df_element

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


,gid,초등학생_인구,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사57ab56ab,153.0,127.017082,37.605811,POINT (127.01708 37.60581),1129066000,성북구,길음제1동
1,다사57ab56ba,430.0,127.017067,37.608064,POINT (127.01707 37.60806),1129066000,성북구,길음제1동
2,다사57ab56bb,349.0,127.017053,37.610318,POINT (127.01705 37.61032),1129066000,성북구,길음제1동
3,다사57ab57aa,330.0,127.017038,37.612571,POINT (127.01704 37.61257),1129066000,성북구,길음제1동
4,다사57ba56aa,277.0,127.019929,37.603569,POINT (127.01993 37.60357),1129066000,성북구,길음제1동
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,49.0,127.036768,37.628423,POINT (127.03677 37.62842),1130560300,강북구,번2동
58,다사59aa59aa,79.0,127.036754,37.630677,POINT (127.03675 37.63068),1130560300,강북구,번2동
59,다사59aa59ab,22.0,127.036740,37.632930,POINT (127.03674 37.63293),1130560300,강북구,번2동
60,다사59ab59aa,86.0,127.039587,37.630688,POINT (127.03959 37.63069),1130560300,강북구,번2동


In [52]:
df_element['행정동'].unique()

array(['길음제1동', '목5동', '번2동'], dtype=object)

In [53]:
df_element.drop_duplicates(['gid'])

,gid,초등학생_인구,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사57ab56ab,153.0,127.017082,37.605811,POINT (127.01708 37.60581),1129066000,성북구,길음제1동
1,다사57ab56ba,430.0,127.017067,37.608064,POINT (127.01707 37.60806),1129066000,성북구,길음제1동
2,다사57ab56bb,349.0,127.017053,37.610318,POINT (127.01705 37.61032),1129066000,성북구,길음제1동
3,다사57ab57aa,330.0,127.017038,37.612571,POINT (127.01704 37.61257),1129066000,성북구,길음제1동
4,다사57ba56aa,277.0,127.019929,37.603569,POINT (127.01993 37.60357),1129066000,성북구,길음제1동
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,49.0,127.036768,37.628423,POINT (127.03677 37.62842),1130560300,강북구,번2동
58,다사59aa59aa,79.0,127.036754,37.630677,POINT (127.03675 37.63068),1130560300,강북구,번2동
59,다사59aa59ab,22.0,127.036740,37.632930,POINT (127.03674 37.63293),1130560300,강북구,번2동
60,다사59ab59aa,86.0,127.039587,37.630688,POINT (127.03959 37.63069),1130560300,강북구,번2동


In [54]:
df_element.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_초등학생_격자250.geojson', driver='GeoJSON')

### 좌표위치 확인

In [55]:
# Train 데이터 좌표 위치 
center = (37.587195,	126.889779)
zoom = 11

dong_geo['색깔'] = 1

m = folium.Map(location=center, zoom_start=zoom)
folium.Choropleth(
    geo_data = dong_geo,
    data = dong_geo,
    columns = ('읍면동명', '색깔'),
    key_on = 'feature.properties.읍면동명',
    fill_color = 'BuPu',
    legend_name = '색깔'
).add_to(m)

# 행정동 naming 
for index, row in dong_geo.iterrows():
    title = f"{row['시군구명']} - {row['읍면동명']}"
    folium.CircleMarker([row['위도'], row['경도']], popup=title, radius =1,color = 'purple').add_to(m)

# for index, row in df_element.iterrows():
#     folium.Marker([row['위도'], row['경도']]).add_to(m)   
for index, row in df_element.iterrows():  
    folium.Circle([row['위도'], row['경도']],
                  radius = 3, color = 'red' ).add_to(m)
m

## 30대

### utm-k(grs80) -> wgs84

In [56]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

# Projection 정의
# UTM-K
proj_UTMK = Proj(init='epsg:5179') #GRS80 UTMK

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

# centroid 찍고 X, Y 반환
pop_30['utm_Y'] = pop_30['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[1]) #geometry에서 utm_Y 반환 
pop_30['utm_Y'] = pop_30['utm_Y'].map(mapper)

pop_30['utm_X'] = pop_30['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[0]) #geometry에서 utm_X 반환 
pop_30['utm_X'] = pop_30['utm_X'].map(mapper)

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['utm_X'], df['utm_Y']), index=['utm_X', 'utm_Y'])


pop_30[['경도', '위도']] = pop_30.apply(transform_utmk_to_w84, axis=1)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [57]:
pop_30 = pop_30.drop(['lbl', 'utm_Y', 'utm_X', 'geometry'], axis=1)
pop_30['val'] = pop_30['val'].fillna(0)
pop_30

,gid,val,경도,위도
0,다사53ba58bb,0.0,126.974439,37.628162
1,다사53ba59aa,0.0,126.974423,37.630416
2,다사53ba59ab,0.0,126.974408,37.632669
3,다사53bb55ab,0.0,126.977493,37.596629
4,다사53bb55ba,0.0,126.977477,37.598882
...,...,...,...,...
1253,다사59bb58ab,243.0,127.045295,37.623950
1254,다사59bb58ba,202.0,127.045281,37.626203
1255,다사59bb58bb,11.0,127.045267,37.628457
1258,다사60aa58ba,71.0,127.048114,37.626214


### 행정동 mapping

In [58]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(pop_30['경도'], pop_30['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
pop_30 = gpd.GeoDataFrame(pop_30, crs=crs, geometry=geometry)
pop_30

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,gid,val,경도,위도,geometry
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816)
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042)
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267)
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663)
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888)
...,...,...,...,...,...
1253,다사59bb58ab,243.0,127.045295,37.623950,POINT (127.04529 37.62395)
1254,다사59bb58ba,202.0,127.045281,37.626203,POINT (127.04528 37.62620)
1255,다사59bb58bb,11.0,127.045267,37.628457,POINT (127.04527 37.62846)
1258,다사60aa58ba,71.0,127.048114,37.626214,POINT (127.04811 37.62621)


In [59]:
# 행정동 포함여부 확인 
pop_30['행정동코드'] = np.zeros(len(pop_30))
pop_30['시군구명'] = np.zeros(len(pop_30))
pop_30['행정동'] = np.zeros(len(pop_30))

for i in tqdm(pop_30.index):
    for j in dong_geo.index:
        if pop_30.geometry[i].within(dong_geo.geometry[j]):
            pop_30['행정동'][i] = dong_geo.읍면동명[j]
            pop_30['행정동코드'][i] = dong_geo.행정동코드[j]
            pop_30['시군구명'][i] = dong_geo.시군구명[j]
            
pop_30['행정동코드'] = pop_30['행정동코드'].astype(int)
pop_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path whi

,gid,val,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816),1111056000,종로구,평창동
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042),1111056000,종로구,평창동
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267),0,0,0
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663),1111056000,종로구,평창동
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888),1111056000,종로구,평창동
...,...,...,...,...,...,...,...,...
1253,다사59bb58ab,243.0,127.045295,37.623950,POINT (127.04529 37.62395),1130560800,강북구,번3동
1254,다사59bb58ba,202.0,127.045281,37.626203,POINT (127.04528 37.62620),1130560800,강북구,번3동
1255,다사59bb58bb,11.0,127.045267,37.628457,POINT (127.04527 37.62846),1130560800,강북구,번3동
1258,다사60aa58ba,71.0,127.048114,37.626214,POINT (127.04811 37.62621),1130560800,강북구,번3동


In [ ]:
print(len(pop_30[pop_30['행정동코드'] == 1129066000]) + len(pop_30[pop_30['행정동코드'] == 1130560300]) + len(pop_30[pop_30['행정동코드'] == 1147055000])) 
len(pop_30[pop_30['행정동코드'] == 1129066000]) , len(pop_30[pop_30['행정동코드'] == 1130560300]) , len(pop_30[pop_30['행정동코드'] == 1147055000])

81


(13, 46, 22)

In [61]:
df_pop_30 = pop_30.copy()
mask1 = df_pop_30['행정동']==0 
mask2 = df_pop_30['행정동코드'] == 1129066000 #'길음1동'
mask3 = df_pop_30['행정동코드'] == 1130560300 #'번2동'
mask4 = df_pop_30['행정동코드'] == 1147055000 #'목5동'

df_pop_30 = df_pop_30[~mask1]
df_pop_30 = df_pop_30[mask2 | mask3 | mask4].reset_index().drop(['index'], axis=1)
df_pop_30.columns = ['gid', '30대_인구', '경도', '위도', 'geometry', '행정동코드','시군구명','행정동']
df_pop_30

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


,gid,30대_인구,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사57ab56ab,422.0,127.017082,37.605811,POINT (127.01708 37.60581),1129066000,성북구,길음제1동
1,다사57ab56ba,570.0,127.017067,37.608064,POINT (127.01707 37.60806),1129066000,성북구,길음제1동
2,다사57ab56bb,350.0,127.017053,37.610318,POINT (127.01705 37.61032),1129066000,성북구,길음제1동
3,다사57ab57aa,255.0,127.017038,37.612571,POINT (127.01704 37.61257),1129066000,성북구,길음제1동
4,다사57ba56aa,472.0,127.019929,37.603569,POINT (127.01993 37.60357),1129066000,성북구,길음제1동
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,182.0,127.036768,37.628423,POINT (127.03677 37.62842),1130560300,강북구,번2동
58,다사59aa59aa,182.0,127.036754,37.630677,POINT (127.03675 37.63068),1130560300,강북구,번2동
59,다사59aa59ab,86.0,127.036740,37.632930,POINT (127.03674 37.63293),1130560300,강북구,번2동
60,다사59ab59aa,256.0,127.039587,37.630688,POINT (127.03959 37.63069),1130560300,강북구,번2동


In [62]:
df_pop_30['행정동'].unique()

array(['길음제1동', '목5동', '번2동'], dtype=object)

In [63]:
df_pop_30.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_30대_격자250.geojson', driver='GeoJSON')

## 40대

In [64]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

# Projection 정의
# UTM-K
proj_UTMK = Proj(init='epsg:5179') #GRS80 UTMK

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

# centroid 찍고 X, Y 반환
pop_40['utm_Y'] = pop_40['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[1]) #geometry에서 utm_Y 반환 
pop_40['utm_Y'] = pop_40['utm_Y'].map(mapper)

pop_40['utm_X'] = pop_40['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[0]) #geometry에서 utm_X 반환 
pop_40['utm_X'] = pop_40['utm_X'].map(mapper)

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['utm_X'], df['utm_Y']), index=['utm_X', 'utm_Y'])


pop_40[['경도', '위도']] = pop_40.apply(transform_utmk_to_w84, axis=1)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [65]:
pop_40 = pop_40.drop(['lbl', 'utm_Y', 'utm_X', 'geometry'], axis=1)
pop_40['val'] = pop_40['val'].fillna(0)
pop_40

,gid,val,경도,위도
0,다사53ba58bb,0.0,126.974439,37.628162
1,다사53ba59aa,0.0,126.974423,37.630416
2,다사53ba59ab,0.0,126.974408,37.632669
3,다사53bb55ab,0.0,126.977493,37.596629
4,다사53bb55ba,0.0,126.977477,37.598882
...,...,...,...,...
1253,다사59bb58ab,339.0,127.045295,37.623950
1254,다사59bb58ba,424.0,127.045281,37.626203
1255,다사59bb58bb,53.0,127.045267,37.628457
1258,다사60aa58ba,122.0,127.048114,37.626214


### 행정동 mapping

In [66]:
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry,색깔
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",1
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ...",1
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ...",1
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ...",1
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ...",1
...,...,...,...,...,...,...,...
419,1174069000,강동구,둔촌제1동,37.523145,127.140435,"MULTIPOLYGON (((127.14580 37.52199, 127.14564 ...",1
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ...",1
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ...",1
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ...",1


In [67]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(pop_40['경도'], pop_40['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
pop_40 = gpd.GeoDataFrame(pop_40, crs=crs, geometry=geometry)
pop_40

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,gid,val,경도,위도,geometry
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816)
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042)
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267)
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663)
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888)
...,...,...,...,...,...
1253,다사59bb58ab,339.0,127.045295,37.623950,POINT (127.04529 37.62395)
1254,다사59bb58ba,424.0,127.045281,37.626203,POINT (127.04528 37.62620)
1255,다사59bb58bb,53.0,127.045267,37.628457,POINT (127.04527 37.62846)
1258,다사60aa58ba,122.0,127.048114,37.626214,POINT (127.04811 37.62621)


In [68]:
# 행정동 포함여부 확인 
pop_40['행정동코드'] = np.zeros(len(pop_40))
pop_40['시군구명'] = np.zeros(len(pop_40))
pop_40['행정동'] = np.zeros(len(pop_40))

for i in tqdm(pop_40.index):
    for j in dong_geo.index:
        if pop_40.geometry[i].within(dong_geo.geometry[j]):
            pop_40['행정동'][i] = dong_geo.읍면동명[j]
            pop_40['행정동코드'][i] = dong_geo.행정동코드[j]
            pop_40['시군구명'][i] = dong_geo.시군구명[j]
            
pop_40['행정동코드'] = pop_40['행정동코드'].astype(int)
pop_40

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path whi

,gid,val,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816),1111056000,종로구,평창동
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042),1111056000,종로구,평창동
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267),0,0,0
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663),1111056000,종로구,평창동
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888),1111056000,종로구,평창동
...,...,...,...,...,...,...,...,...
1253,다사59bb58ab,339.0,127.045295,37.623950,POINT (127.04529 37.62395),1130560800,강북구,번3동
1254,다사59bb58ba,424.0,127.045281,37.626203,POINT (127.04528 37.62620),1130560800,강북구,번3동
1255,다사59bb58bb,53.0,127.045267,37.628457,POINT (127.04527 37.62846),1130560800,강북구,번3동
1258,다사60aa58ba,122.0,127.048114,37.626214,POINT (127.04811 37.62621),1130560800,강북구,번3동


In [69]:
df_pop_40 = pop_40.copy()
mask1 = df_pop_40['행정동']==0 
mask2 = df_pop_40['행정동코드'] == 1129066000 #'길음1동'
mask3 = df_pop_40['행정동코드'] == 1130560300 #'번2동'
mask4 = df_pop_40['행정동코드'] == 1147055000 #'목5동'

df_pop_40 = df_pop_40[~mask1]
df_pop_40 = df_pop_40[mask2 | mask3 | mask4].reset_index().drop(['index'], axis=1)
df_pop_40.columns = ['gid', '40대_인구', '경도', '위도', 'geometry', '행정동코드','시군구명','행정동']
df_pop_40

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


,gid,40대_인구,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사57ab56ab,530.0,127.017082,37.605811,POINT (127.01708 37.60581),1129066000,성북구,길음제1동
1,다사57ab56ba,948.0,127.017067,37.608064,POINT (127.01707 37.60806),1129066000,성북구,길음제1동
2,다사57ab56bb,737.0,127.017053,37.610318,POINT (127.01705 37.61032),1129066000,성북구,길음제1동
3,다사57ab57aa,756.0,127.017038,37.612571,POINT (127.01704 37.61257),1129066000,성북구,길음제1동
4,다사57ba56aa,783.0,127.019929,37.603569,POINT (127.01993 37.60357),1129066000,성북구,길음제1동
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,238.0,127.036768,37.628423,POINT (127.03677 37.62842),1130560300,강북구,번2동
58,다사59aa59aa,215.0,127.036754,37.630677,POINT (127.03675 37.63068),1130560300,강북구,번2동
59,다사59aa59ab,191.0,127.036740,37.632930,POINT (127.03674 37.63293),1130560300,강북구,번2동
60,다사59ab59aa,338.0,127.039587,37.630688,POINT (127.03959 37.63069),1130560300,강북구,번2동


In [70]:
df_pop_40['행정동'].unique()

array(['길음제1동', '목5동', '번2동'], dtype=object)

In [71]:
df_pop_40.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_40대_격자250.geojson', driver='GeoJSON')

## 전체인구

In [72]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

# Projection 정의
# UTM-K
proj_UTMK = Proj(init='epsg:5179') #GRS80 UTMK

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

# centroid 찍고 X, Y 반환
pop_all['utm_Y'] = pop_all['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[1]) #geometry에서 utm_Y 반환 
pop_all['utm_Y'] = pop_all['utm_Y'].map(mapper)

pop_all['utm_X'] = pop_all['geometry'].centroid
mapper = lambda x: float(str(x)[6:].strip('\(').strip('\)').split(' ')[0]) #geometry에서 utm_X 반환 
pop_all['utm_X'] = pop_all['utm_X'].map(mapper)

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['utm_X'], df['utm_Y']), index=['utm_X', 'utm_Y'])


pop_all[['경도', '위도']] = pop_all.apply(transform_utmk_to_w84, axis=1)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [73]:
pop_all = pop_all.drop(['lbl', 'utm_Y', 'utm_X', 'geometry'], axis=1)
pop_all['val'] = pop_all['val'].fillna(0)
pop_all

,gid,val,경도,위도
0,다사53ba58bb,0.0,126.974439,37.628162
1,다사53ba59aa,0.0,126.974423,37.630416
2,다사53ba59ab,0.0,126.974408,37.632669
3,다사53bb55ab,0.0,126.977493,37.596629
4,다사53bb55ba,0.0,126.977477,37.598882
...,...,...,...,...
1253,다사59bb58ab,1235.0,127.045295,37.623950
1254,다사59bb58ba,1735.0,127.045281,37.626203
1255,다사59bb58bb,259.0,127.045267,37.628457
1258,다사60aa58ba,558.0,127.048114,37.626214


### 행정동 mapping

In [74]:
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry,색깔
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",1
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ...",1
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ...",1
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ...",1
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ...",1
...,...,...,...,...,...,...,...
419,1174069000,강동구,둔촌제1동,37.523145,127.140435,"MULTIPOLYGON (((127.14580 37.52199, 127.14564 ...",1
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ...",1
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ...",1
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ...",1


In [75]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(pop_all['경도'], pop_all['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
pop_all = gpd.GeoDataFrame(pop_all, crs=crs, geometry=geometry)
pop_all

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,gid,val,경도,위도,geometry
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816)
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042)
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267)
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663)
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888)
...,...,...,...,...,...
1253,다사59bb58ab,1235.0,127.045295,37.623950,POINT (127.04529 37.62395)
1254,다사59bb58ba,1735.0,127.045281,37.626203,POINT (127.04528 37.62620)
1255,다사59bb58bb,259.0,127.045267,37.628457,POINT (127.04527 37.62846)
1258,다사60aa58ba,558.0,127.048114,37.626214,POINT (127.04811 37.62621)


In [76]:
# 행정동 포함여부 확인 
pop_all['행정동코드'] = np.zeros(len(pop_all))
pop_all['시군구명'] = np.zeros(len(pop_all))
pop_all['행정동'] = np.zeros(len(pop_all))

for i in tqdm(pop_all.index):
    for j in dong_geo.index:
        if pop_all.geometry[i].within(dong_geo.geometry[j]):
            pop_all['행정동'][i] = dong_geo.읍면동명[j]
            pop_all['행정동코드'][i] = dong_geo.행정동코드[j]
            pop_all['시군구명'][i] = dong_geo.시군구명[j]
            
pop_all['행정동코드'] = pop_all['행정동코드'].astype(int)
pop_all

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path whi

,gid,val,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사53ba58bb,0.0,126.974439,37.628162,POINT (126.97444 37.62816),1111056000,종로구,평창동
1,다사53ba59aa,0.0,126.974423,37.630416,POINT (126.97442 37.63042),1111056000,종로구,평창동
2,다사53ba59ab,0.0,126.974408,37.632669,POINT (126.97441 37.63267),0,0,0
3,다사53bb55ab,0.0,126.977493,37.596629,POINT (126.97749 37.59663),1111056000,종로구,평창동
4,다사53bb55ba,0.0,126.977477,37.598882,POINT (126.97748 37.59888),1111056000,종로구,평창동
...,...,...,...,...,...,...,...,...
1253,다사59bb58ab,1235.0,127.045295,37.623950,POINT (127.04529 37.62395),1130560800,강북구,번3동
1254,다사59bb58ba,1735.0,127.045281,37.626203,POINT (127.04528 37.62620),1130560800,강북구,번3동
1255,다사59bb58bb,259.0,127.045267,37.628457,POINT (127.04527 37.62846),1130560800,강북구,번3동
1258,다사60aa58ba,558.0,127.048114,37.626214,POINT (127.04811 37.62621),1130560800,강북구,번3동


In [77]:
df_pop_all = pop_all.copy()
mask1 = df_pop_all['행정동']==0 
mask2 = df_pop_all['행정동코드'] == 1129066000 #'길음1동'
mask3 = df_pop_all['행정동코드'] == 1130560300 #'번2동'
mask4 = df_pop_all['행정동코드'] == 1147055000 #'목5동'

df_pop_all = df_pop_all[~mask1]
df_pop_all = df_pop_all[mask2 | mask3 | mask4].reset_index().drop(['index'], axis=1)
df_pop_all.columns = ['gid', '전체_인구', '경도', '위도', 'geometry', '행정동코드','시군구명','행정동']
df_pop_all

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


,gid,전체_인구,경도,위도,geometry,행정동코드,시군구명,행정동
0,다사57ab56ab,2074.0,127.017082,37.605811,POINT (127.01708 37.60581),1129066000,성북구,길음제1동
1,다사57ab56ba,2876.0,127.017067,37.608064,POINT (127.01707 37.60806),1129066000,성북구,길음제1동
2,다사57ab56bb,2205.0,127.017053,37.610318,POINT (127.01705 37.61032),1129066000,성북구,길음제1동
3,다사57ab57aa,2353.0,127.017038,37.612571,POINT (127.01704 37.61257),1129066000,성북구,길음제1동
4,다사57ba56aa,2759.0,127.019929,37.603569,POINT (127.01993 37.60357),1129066000,성북구,길음제1동
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1000.0,127.036768,37.628423,POINT (127.03677 37.62842),1130560300,강북구,번2동
58,다사59aa59aa,1093.0,127.036754,37.630677,POINT (127.03675 37.63068),1130560300,강북구,번2동
59,다사59aa59ab,822.0,127.036740,37.632930,POINT (127.03674 37.63293),1130560300,강북구,번2동
60,다사59ab59aa,1407.0,127.039587,37.630688,POINT (127.03959 37.63069),1130560300,강북구,번2동


In [78]:
df_pop_all['행정동'].unique()

array(['길음제1동', '목5동', '번2동'], dtype=object)

In [79]:
df_pop_all.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_전체인구_격자250.geojson', driver='GeoJSON')

## 학교 주변 881m 

In [80]:
df_element = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_초등학생_격자250.geojson', driver='GeoJSON')
df_pop_30 = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_30대_격자250.geojson', driver='GeoJSON')
df_pop_40 = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_40대_격자250.geojson', driver='GeoJSON')
df_pop_all = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/1109_동3개_전체인구_격자250.geojson', driver='GeoJSON')

In [81]:
df_school = pd.read_pickle('/content/drive/My Drive/Project/Seoul/초등학교/초등학교.pickle')
df_school

,학교명,구,행정동코드,행정동,소재지지번주소,소재지도로명주소,위도,경도,geometry
0,서울휘경초등학교,동대문구,1123073000,휘경제2동,서울특별시 동대문구 휘경동 302-2,서울특별시 동대문구 망우로6길 48,37.585736,127.058213,POINT (127.05821 37.58574)
1,서울동답초등학교,동대문구,1123061000,답십리제2동,서울특별시 동대문구 답십리동 2-24,서울특별시 동대문구 답십리로60길 12,37.571503,127.064034,POINT (127.06403 37.57150)
2,서울답십리초등학교,동대문구,1123060000,답십리제1동,서울특별시 동대문구 답십리동 274,서울특별시 동대문구 전농로3길 23,37.568804,127.055429,POINT (127.05543 37.56880)
3,서울신답초등학교,동대문구,1123060000,답십리제1동,서울특별시 동대문구 답십리동 463-13,서울특별시 동대문구 천호대로 177,37.572705,127.042962,POINT (127.04296 37.57271)
4,서울용두초등학교,동대문구,1123053600,용신동,서울특별시 동대문구 용두동 237-1,서울특별시 동대문구 천호대로13길 43,37.576415,127.028387,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...,...
598,서울위례별초등학교,송파구,1171064700,위례동,서울특별시 송파구 장지동 886,서울특별시 송파구 위례광장로 243,37.479086,127.145042,POINT (127.14504 37.47909)
599,서울하늘숲초등학교,구로구,1153078000,오류제2동,서울특별시 구로구 천왕동 산13-1,서울특별시 구로구 오리로 1100,37.483405,126.842135,POINT (126.84213 37.48340)
600,서울언남초등학교,서초구,1165066000,내곡동,서울특별시 서초구 내곡동 121,서울특별시 서초구 헌릉로8길 42,37.453898,127.061885,POINT (127.06189 37.45390)
601,서울항동초등학교,구로구,1153078000,오류제2동,서울특별시 구로구 항동 160,서울특별시 구로구 연동로 178,37.476972,126.824078,POINT (126.82408 37.47697)


In [82]:
# 현재 각 학교주변 있는 좌표들 구하기 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 
from tqdm.notebook import tqdm

def point_school(df_pop, df_school):
  lscp = []

  for k in tqdm(range(len(df_pop))): 
    gid_nm = df_pop['gid'].iloc[k]
    x = (df_pop['위도'].iloc[k], df_pop['경도'].iloc[k])
    for i in range(len(df_school)): 
      y = (df_school['위도'].iloc[i], df_school['경도'].iloc[i])
      h = haversine(x,y, unit='m') 
      if h <= 881:  #전체 좌표 중에서 초등학교 881m 있는 것만 필터링
        lscp.append([gid_nm, df_school['학교명'].iloc[i], h])
  
  return lscp 

#### 초등학생 인구_881m

In [83]:
lscp_element = point_school(df_pop_30, df_school)
lscp_element

[['다사57ab56ab', '서울숭덕초등학교', 400.7932064551218],
 ['다사57ab56ab', '서울길원초등학교', 362.7224210613397],
 ['다사57ab56ab', '서울길음초등학교', 870.3972632976388],
 ['다사57ab56ab', '서울미아초등학교', 666.2531250097295],
 ['다사57ab56ab', '서울정수초등학교', 855.2960268038003],
 ['다사57ab56ba', '서울숭덕초등학교', 577.5685953239667],
 ['다사57ab56ba', '서울길원초등학교', 113.8044296344176],
 ['다사57ab56ba', '서울길음초등학교', 620.088942182714],
 ['다사57ab56ba', '서울미아초등학교', 491.39199534216067],
 ['다사57ab56bb', '서울숭덕초등학교', 795.0439445821679],
 ['다사57ab56bb', '서울길원초등학교', 141.04821133767896],
 ['다사57ab56bb', '서울길음초등학교', 370.1196412866157],
 ['다사57ab56bb', '서울미아초등학교', 405.70716528725734],
 ['다사57ab56bb', '서울삼각산초등학교', 656.3021075651408],
 ['다사57ab56bb', '서울송천초등학교', 778.992232541739],
 ['다사57ab57aa', '서울길원초등학교', 390.3804038393599],
 ['다사57ab57aa', '서울길음초등학교', 122.5806253689602],
 ['다사57ab57aa', '서울미아초등학교', 461.83299474403145],
 ['다사57ab57aa', '서울삼각산초등학교', 414.64487648379566],
 ['다사57ab57aa', '서울송천초등학교', 644.5417522129022],
 ['다사57ba56aa', '서울정덕초등학교', 824.570

In [84]:
#좌표별로 groupby 
df_loc = pd.DataFrame(lscp_element, columns = ['gid', '학교명', 'haversine']) 
lscp_element_groupby = dict(list(df_loc.groupby('gid')))

# 전체 좌표 리스트를 데이터 프레임으로 만들기 
df_point = pd.DataFrame(lscp_element_groupby.items())
df_point['학교명'] = np.array(np.nan)

# 각 좌표의 가까운'학교명 리스트를 데이터 프레임으로 만들기 
schools = []
for idx, k in enumerate(lscp_element_groupby.keys()):
  schools.append(lscp_element_groupby[k]['학교명'].values)

#좌표와 각 좌표별'학교명 데이터 프레임 합치기 
df_point['학교명'] = schools
df_point = df_point[[0,'학교명']]
df_point.columns = ['gid', '학교명']
df_point

,gid,학교명
0,다사44aa48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
1,다사44ab48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
2,다사44ab48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
3,다사44ab48ba,"[서울정목초등학교, 서울월촌초등학교, 서울영도초등학교]"
4,다사44ba48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
...,...,...
57,다사59aa58bb,"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]"
58,다사59aa59aa,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]"
59,다사59aa59ab,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]"
60,다사59ab59aa,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]"


In [85]:
df_point['학교개수'] = df_point['학교명'].apply(lambda x: len(x))
df_point

,gid,학교명,학교개수
0,다사44aa48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",4
1,다사44ab48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",4
2,다사44ab48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",3
3,다사44ab48ba,"[서울정목초등학교, 서울월촌초등학교, 서울영도초등학교]",3
4,다사44ba48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",3
...,...,...,...
57,다사59aa58bb,"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]",3
58,다사59aa59aa,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4
59,다사59aa59ab,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4
60,다사59ab59aa,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]",3


### merge

In [86]:
df1 = df_element.merge(df_point, how = 'left', on='gid')
df2 = df1.merge(df_pop_30[['gid', '30대_인구']],how ='left',  on='gid')
df3 = df2.merge(df_pop_40[['gid', '40대_인구']], how ='left',on = 'gid')
df4 = df3.merge(df_pop_all[['gid', '전체_인구']],how ='left', on = 'gid')
df4

,gid,초등학생_인구,경도,위도,행정동코드,시군구명,행정동,geometry,학교명,학교개수,30대_인구,40대_인구,전체_인구
0,다사57ab56ab,153.0,127.017082,37.605811,1129066000,성북구,길음제1동,POINT (127.01708 37.60581),"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울정수초...",5,422.0,530.0,2074.0
1,다사57ab56ba,430.0,127.017067,37.608064,1129066000,성북구,길음제1동,POINT (127.01707 37.60806),"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교]",4,570.0,948.0,2876.0
2,다사57ab56bb,349.0,127.017053,37.610318,1129066000,성북구,길음제1동,POINT (127.01705 37.61032),"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산...",6,350.0,737.0,2205.0
3,다사57ab57aa,330.0,127.017038,37.612571,1129066000,성북구,길음제1동,POINT (127.01704 37.61257),"[서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산초등학교, 서울송천...",5,255.0,756.0,2353.0
4,다사57ba56aa,277.0,127.019929,37.603569,1129066000,성북구,길음제1동,POINT (127.01993 37.60357),"[서울정덕초등학교, 서울숭덕초등학교, 서울길원초등학교, 서울미아초등학교, 서울개운초...",6,472.0,783.0,2759.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,다사59aa58bb,49.0,127.036768,37.628423,1130560300,강북구,번2동,POINT (127.03677 37.62842),"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]",3,182.0,238.0,1000.0
58,다사59aa59aa,79.0,127.036754,37.630677,1130560300,강북구,번2동,POINT (127.03675 37.63068),"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,182.0,215.0,1093.0
59,다사59aa59ab,22.0,127.036740,37.632930,1130560300,강북구,번2동,POINT (127.03674 37.63293),"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,86.0,191.0,822.0
60,다사59ab59aa,86.0,127.039587,37.630688,1130560300,강북구,번2동,POINT (127.03959 37.63069),"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]",3,256.0,338.0,1407.0


In [87]:
df_pop = df4[['gid', '행정동코드','시군구명','행정동','학교명', '학교개수', '초등학생_인구','30대_인구', '40대_인구', '전체_인구','경도', '위도', 'geometry']]
df_pop

,gid,행정동코드,시군구명,행정동,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,경도,위도,geometry
0,다사57ab56ab,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울정수초...",5,153.0,422.0,530.0,2074.0,127.017082,37.605811,POINT (127.01708 37.60581)
1,다사57ab56ba,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교]",4,430.0,570.0,948.0,2876.0,127.017067,37.608064,POINT (127.01707 37.60806)
2,다사57ab56bb,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산...",6,349.0,350.0,737.0,2205.0,127.017053,37.610318,POINT (127.01705 37.61032)
3,다사57ab57aa,1129066000,성북구,길음제1동,"[서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산초등학교, 서울송천...",5,330.0,255.0,756.0,2353.0,127.017038,37.612571,POINT (127.01704 37.61257)
4,다사57ba56aa,1129066000,성북구,길음제1동,"[서울정덕초등학교, 서울숭덕초등학교, 서울길원초등학교, 서울미아초등학교, 서울개운초...",6,277.0,472.0,783.0,2759.0,127.019929,37.603569,POINT (127.01993 37.60357)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,강북구,번2동,"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]",3,49.0,182.0,238.0,1000.0,127.036768,37.628423,POINT (127.03677 37.62842)
58,다사59aa59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,79.0,182.0,215.0,1093.0,127.036754,37.630677,POINT (127.03675 37.63068)
59,다사59aa59ab,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,22.0,86.0,191.0,822.0,127.036740,37.632930,POINT (127.03674 37.63293)
60,다사59ab59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]",3,86.0,256.0,338.0,1407.0,127.039587,37.630688,POINT (127.03959 37.63069)


In [88]:
df_pop.to_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_인구.pickle')

In [142]:
df_pop.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_인구.csv')

## 시각화


In [89]:
mask1 = dong_geo['행정동코드']==1130560300
mask2 = dong_geo['행정동코드']==1147055000 
mask3 = dong_geo['행정동코드']==1129066000

target_geo = dong_geo[mask1|mask2|mask3]
target_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry,색깔
118,1129066000,성북구,길음제1동,37.608112,127.019735,"MULTIPOLYGON (((127.02203 37.61229, 127.02223 ...",1
131,1130560300,강북구,번2동,37.629515,127.034795,"MULTIPOLYGON (((127.03848 37.63391, 127.04148 ...",1
238,1147055000,양천구,목5동,37.536311,126.881027,"MULTIPOLYGON (((126.88516 37.54359, 126.88617 ...",1


In [90]:
# 동 3개 초등학교 인구 
target_ele_pop = df_pop.groupby(['행정동코드','행정동'],as_index=False).agg('sum').drop(['경도', '위도'],axis=1)
target_ele_pop = target_ele_pop.merge(df_pop[['행정동코드', '위도', '경도']], on='행정동코드')
target_ele_pop

,행정동코드,행정동,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,위도,경도
0,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.605811,127.017082
1,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.608064,127.017067
2,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.610318,127.017053
3,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.612571,127.017038
4,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.603569,127.019929
...,...,...,...,...,...,...,...,...,...
57,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.537608,126.887353
58,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.539861,126.887335
59,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.530863,126.890238
60,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.533116,126.890219


In [92]:
# 은평구 초등학교 위치 (+ 마포구)
mask1 = df_school['행정동코드']==1130560300
mask2 = df_school['행정동코드']==1147055000 
mask3 = df_school['행정동코드']==1129066000

target_school = df_school[mask1|mask2|mask3]
target_school

,학교명,구,행정동코드,행정동,소재지지번주소,소재지도로명주소,위도,경도,geometry
344,서울목원초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 191-1,서울특별시 양천구 목동중앙로 204,37.542920,126.882446,POINT (126.88245 37.54292)
345,서울경인초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 911-4,서울특별시 양천구 안양천로 1009,37.532870,126.885698,POINT (126.88570 37.53287)
372,서울영도초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 903-1,서울특별시 양천구 목동중앙로 70,37.535963,126.871927,POINT (126.87193 37.53596)
546,서울길원초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 1280-4,서울특별시 성북구 길음로15길 55,37.609065,127.016798,POINT (127.01680 37.60907)
550,서울길음초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 635-7,서울특별시 성북구 길음로 142,37.613631,127.016659,POINT (127.01666 37.61363)
551,서울미아초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 1168,서울특별시 성북구 삼양로 77,37.610589,127.021646,POINT (127.02165 37.61059)


In [95]:
# Train 데이터 좌표 위치 
center = (37.467548,	126.890947)
zoom = 12

m = folium.Map(location=center, zoom_start=zoom, tiles='cartodbpositron')
folium.Choropleth(
    geo_data = target_geo,
    data = target_ele_pop,
    columns = ('행정동', '초등학생_인구'),
    key_on = 'feature.properties.읍면동명',
    fill_color = 'Blues',
    legend_name = '초등학생_인구'
).add_to(m)

# 행정동 naming 
for index, row in target_geo.iterrows():
    title = f"{row['시군구명']} - {row['읍면동명']}"
    folium.CircleMarker([row['위도'], row['경도']], popup=title, radius =1,color = 'white').add_to(m)

# 초등학교 위치
for index, row in target_school.iterrows():
    title = f"{row['학교명']}"
    folium.Marker([row['위도'], row['경도']],popup=title, icon=folium.Icon(icon ='dumbell', color = 'orange')).add_to(m)  
    folium.Circle([row['위도'], row['경도']], radius =881, fill='#ffa5a5', color = '#ffa5a5').add_to(m)

# # 전체 target 좌표
# for index, row in pop_30.iterrows():  
#     folium.Circle([row['위도'], row['경도']],
#                   radius = 3, color = 'red' ).add_to(m)   
# 행정동 내 좌표 
for index, row in df_pop.iterrows():  
    folium.Circle([row['위도'], row['경도']],
                  radius = 3, color = 'blue' ).add_to(m)
               
m

## 대체시설

In [96]:
df_pop = pd.read_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_인구.pickle')
df_pop

,gid,행정동코드,시군구명,행정동,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,경도,위도,geometry
0,다사57ab56ab,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울정수초...",5,153.0,422.0,530.0,2074.0,127.017082,37.605811,POINT (127.01708 37.60581)
1,다사57ab56ba,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교]",4,430.0,570.0,948.0,2876.0,127.017067,37.608064,POINT (127.01707 37.60806)
2,다사57ab56bb,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산...",6,349.0,350.0,737.0,2205.0,127.017053,37.610318,POINT (127.01705 37.61032)
3,다사57ab57aa,1129066000,성북구,길음제1동,"[서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산초등학교, 서울송천...",5,330.0,255.0,756.0,2353.0,127.017038,37.612571,POINT (127.01704 37.61257)
4,다사57ba56aa,1129066000,성북구,길음제1동,"[서울정덕초등학교, 서울숭덕초등학교, 서울길원초등학교, 서울미아초등학교, 서울개운초...",6,277.0,472.0,783.0,2759.0,127.019929,37.603569,POINT (127.01993 37.60357)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,강북구,번2동,"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]",3,49.0,182.0,238.0,1000.0,127.036768,37.628423,POINT (127.03677 37.62842)
58,다사59aa59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,79.0,182.0,215.0,1093.0,127.036754,37.630677,POINT (127.03675 37.63068)
59,다사59aa59ab,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,22.0,86.0,191.0,822.0,127.036740,37.632930,POINT (127.03674 37.63293)
60,다사59ab59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]",3,86.0,256.0,338.0,1407.0,127.039587,37.630688,POINT (127.03959 37.63069)


In [97]:
merge_base = df_pop[['gid', '행정동코드', '행정동','위도', '경도']]
merge_base

,gid,행정동코드,행정동,위도,경도
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929
...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587


In [98]:
df_academy = pd.read_pickle('/content/drive/My Drive/Project/Seoul/대체시설/초등보습학원_좌표.pkl')
df_academy.columns = ['학원명', '학원주소', 'hcode', 'hname', '위도', '경도']
df_academy['위도'] = df_academy['위도'].astype(float)
df_academy['경도'] = df_academy['경도'].astype(float)

In [99]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_academy['위도'], df_academy['경도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
df_academy = gpd.GeoDataFrame(df_academy, crs=crs, geometry=geometry)
df_academy

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,학원명,학원주소,hcode,hname,위도,경도,geometry
0,예맥보습학원,서울특별시 중랑구 동일로151길 30 2층3층 (묵동),1126063000,묵제2동,37.608179,127.076350,POINT (37.608 127.076)
1,세진보습학원,서울특별시 중랑구 공릉로 58 2층4층일부 (묵동),1126062000,묵제1동,37.615815,127.079262,POINT (37.616 127.079)
2,대찬하브루타학원,"서울특별시 동대문구 전농로 120 , 3층 (전농동)",1123057000,전농제2동,37.575698,127.057812,POINT (37.576 127.058)
3,장안로드맵수학학원,"서울특별시 동대문구 한천로 124 , 2층 202호 (장안동)",1123065000,장안제1동,37.572103,127.066114,POINT (37.572 127.066)
4,더하이스트수학학원,"서울특별시 동대문구 사가정로 190 전농우성아파트 상가4층7,8,9호 (전농동)",1123057000,전농제2동,37.578340,127.066057,POINT (37.578 127.066)
...,...,...,...,...,...,...,...
4978,헤럴드스쿨어학원,"서울특별시 성북구 종암로 120 , 2층 일부 (종암동)",1129070500,종암동,37.600922,127.033877,POINT (37.601 127.034)
4979,청담강북어학원,"서울특별시 강북구 삼양로27길 35-21 , 5층 501,502,504,505호 (...",1130557500,삼각산동,37.617940,127.018798,POINT (37.618 127.019)
4980,EIE(이아이이)어학원,"서울특별시 강북구 도봉로 189 , 4층,5층 (미아동)",1130553500,미아동,37.625686,127.026165,POINT (37.626 127.026)
4981,성북메가스터디학원,서울특별시 성북구 아리랑로 17 민국빌딩 1-5층 (동소문동6가),1129057500,동선동,37.594424,127.015950,POINT (37.594 127.016)


In [100]:
# 현재 각 좌표 주변 있는 좌표들 구하기 
from tqdm.notebook import tqdm

def point_target(df_point, df_x):
  lscp = []

  for k in tqdm(range(len(df_point))): 
    gid_nm = df_point['gid'].iloc[k]
    x = (df_point['위도'].iloc[k], df_point['경도'].iloc[k])
    for i in range(len(df_x)): 
      y = (df_x['위도'].iloc[i], df_x['경도'].iloc[i])
      h = haversine(x,y, unit='m') 
      if h <= 881:  #전체 좌표 중에서 초등학교 881m 있는 것만 필터링
        lscp.append([gid_nm, df_x['학원명'].iloc[i], h])
  
  return lscp 

In [101]:
lscp_academy = point_target(merge_base, df_academy)
lscp_academy

[['다사57ab56ab', '으뜸보습학원', 344.0111784726108],
 ['다사57ab56ab', '아이즈학원', 319.7987691821349],
 ['다사57ab56ab', '투탑학원', 666.6825737796448],
 ['다사57ab56ab', '남림학원', 622.1976292838107],
 ['다사57ab56ab', '스타트보습학원', 667.2051527932383],
 ['다사57ab56ab', '소마성북학원', 667.2051527932383],
 ['다사57ab56ab', '더웨이(theWay)학원', 411.0578167514424],
 ['다사57ab56ab', '능률주니어랩돈암학원', 746.0597210571857],
 ['다사57ab56ab', '시온보습학원', 648.916234971704],
 ['다사57ab56ab', '청암학원', 767.8191534348011],
 ['다사57ab56ab', '정릉대성학원', 243.81396621224184],
 ['다사57ab56ab', '씨씨엠(CCM)수학영어학원', 599.6709519419987],
 ['다사57ab56ab', '파워리딩영어독서전문학원', 528.7476952075908],
 ['다사57ab56ab', '잉글리쉬무무길음제일학원', 719.6650339784868],
 ['다사57ab56ab', '메가포스학원', 650.0393642401756],
 ['다사57ab56ab', '고대학사학원', 561.9487875077117],
 ['다사57ab56ab', '길음지앤비보습학원', 246.43530384748018],
 ['다사57ab56ab', '마이더스잉글리쉬영어학원', 450.12192006601265],
 ['다사57ab56ab', 'JK정석학원', 507.4693094201485],
 ['다사57ab56ab', '윤선생IGSE길음뉴타운학원', 499.9833845133623],
 ['다사57ab56ab', '멜리사영어전문학원', 475.832

In [102]:
#좌표별로 groupby 
df_loc = pd.DataFrame(lscp_academy, columns = ['gid', '학원명', 'haversine']) 
lscp_academy_groupby = dict(list(df_loc.groupby('gid')))

# 전체 좌표 리스트를 데이터 프레임으로 만들기 
df_point_academy = pd.DataFrame(lscp_academy_groupby.items())
df_point_academy['학원명'] = np.array(np.nan)

# 각 좌표의 가까운'학원명 리스트를 데이터 프레임으로 만들기 
schools = []
for idx, k in enumerate(lscp_academy_groupby.keys()):
  schools.append(lscp_academy_groupby[k]['학원명'].values)

#좌표와 각 좌표별'학원명 데이터 프레임 합치기 
df_point_academy['학원명'] = schools
df_point_academy = df_point_academy[[0,'학원명']]
df_point_academy.columns = ['gid', '학원명']
df_point_academy

,gid,학원명
0,다사44aa48aa,"[길벗아카데미학원, 목동최강학원, 신미래탐구학원, 멘토보습학원, 아드폰테스2관학원,..."
1,다사44ab48aa,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 아드폰테스2관학원, 신관하이씨앤씨학..."
2,다사44ab48ab,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 아드폰테스2관학원, 아카데미리더스학..."
3,다사44ab48ba,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 멘사매쓰학원, 아드폰테스2관학원, ..."
4,다사44ba48ab,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 멘사매쓰학원, 아드폰테스2관학원, ..."
...,...,...
57,다사59aa58bb,"[캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이(SE)&뮤엠영어창동신..."
58,다사59aa59aa,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이..."
59,다사59aa59ab,"[S.E.명문학원, 대성탑(TOP)학원, 박정화뮤엠영어학원, 캠브리지영어학원, 뉴플..."
60,다사59ab59aa,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이..."


In [103]:
df_point_academy['학원개수'] = df_point_academy['학원명'].apply(lambda x: len(x))
df_point_academy

,gid,학원명,학원개수
0,다사44aa48aa,"[길벗아카데미학원, 목동최강학원, 신미래탐구학원, 멘토보습학원, 아드폰테스2관학원,...",101
1,다사44ab48aa,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 아드폰테스2관학원, 신관하이씨앤씨학...",118
2,다사44ab48ab,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 아드폰테스2관학원, 아카데미리더스학...",137
3,다사44ab48ba,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 멘사매쓰학원, 아드폰테스2관학원, ...",125
4,다사44ba48ab,"[길벗아카데미학원, 목동최강학원, 멘토보습학원, 멘사매쓰학원, 아드폰테스2관학원, ...",134
...,...,...,...
57,다사59aa58bb,"[캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이(SE)&뮤엠영어창동신...",14
58,다사59aa59aa,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이...",20
59,다사59aa59ab,"[S.E.명문학원, 대성탑(TOP)학원, 박정화뮤엠영어학원, 캠브리지영어학원, 뉴플...",24
60,다사59ab59aa,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이...",15


In [104]:
df_point_academy = merge_base.merge(df_point_academy, on='gid', how='left')
df_point_academy

,gid,행정동코드,행정동,위도,경도,학원명,학원개수
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082,"[으뜸보습학원, 아이즈학원, 투탑학원, 남림학원, 스타트보습학원, 소마성북학원, 더...",62
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067,"[으뜸보습학원, 아이즈학원, 남림학원, 스타트보습학원, 소마성북학원, 더웨이(the...",54
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053,"[으뜸보습학원, 아이즈학원, 스타트보습학원, 소마성북학원, 더웨이(theWay)학원...",55
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038,"[아이즈학원, 가온수학학원, 스타트보습학원, 소마성북학원, 참조은학원, 강북뉴스터디...",58
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929,"[으뜸보습학원, 아이즈학원, 투탑학원, 남림학원, 스타트보습학원, 소마성북학원, 더...",68
...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768,"[캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이(SE)&뮤엠영어창동신...",14
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이...",20
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740,"[S.E.명문학원, 대성탑(TOP)학원, 박정화뮤엠영어학원, 캠브리지영어학원, 뉴플...",24
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587,"[대성탑(TOP)학원, 캠브리지영어학원, 최윤영어학원, 최윤2관학원, 엠베스트에스이...",15


In [105]:
df_point_academy.to_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_학원정보.pickle')

In [141]:
df_point_academy.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_학원정보.csv')

## 시각화


In [ ]:
mask1 = dong_geo['행정동코드']==1130560300
mask2 = dong_geo['행정동코드']==1147055000 
mask3 = dong_geo['행정동코드']==1129066000

target_geo = dong_geo[mask1|mask2|mask3]
target_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry,색깔
118,1129066000,성북구,길음제1동,37.608112,127.019735,"MULTIPOLYGON (((127.02203 37.61229, 127.02223 ...",1
131,1130560300,강북구,번2동,37.629515,127.034795,"MULTIPOLYGON (((127.03848 37.63391, 127.04148 ...",1
238,1147055000,양천구,목5동,37.536311,126.881027,"MULTIPOLYGON (((126.88516 37.54359, 126.88617 ...",1


In [ ]:
# 동 3개 초등학교 인구 
target_ele_pop = df_pop.groupby(['행정동코드','행정동'],as_index=False).agg('sum').drop(['경도', '위도'],axis=1)
target_ele_pop = target_ele_pop.merge(df_pop[['행정동코드', '위도', '경도']], on='행정동코드')
target_ele_pop

,행정동코드,행정동,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,위도,경도
0,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.605811,127.017082
1,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.608064,127.017067
2,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.610318,127.017053
3,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.612571,127.017038
4,1129066000,길음제1동,66,3282.0,4482.0,7928.0,26068.0,37.603569,127.019929
...,...,...,...,...,...,...,...,...,...
57,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.537608,126.887353
58,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.539861,126.887335
59,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.530863,126.890238
60,1147055000,목5동,118,4156.0,4136.0,9476.0,34650.0,37.533116,126.890219


In [ ]:
# 은평구 초등학교 위치 (+ 마포구)
mask1 = df_school['행정동코드']==1130560300
mask2 = df_school['행정동코드']==1147055000 
mask3 = df_school['행정동코드']==1129066000

target_school = df_school[mask1|mask2|mask3]
target_school

,학교명,구,행정동코드,행정동,소재지지번주소,소재지도로명주소,위도,경도,geometry
344,서울목원초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 191-1,서울특별시 양천구 목동중앙로 204,37.542920,126.882446,POINT (126.88245 37.54292)
345,서울경인초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 911-4,서울특별시 양천구 안양천로 1009,37.532870,126.885698,POINT (126.88570 37.53287)
372,서울영도초등학교,양천구,1147055000,목5동,서울특별시 양천구 목동 903-1,서울특별시 양천구 목동중앙로 70,37.535963,126.871927,POINT (126.87193 37.53596)
546,서울길원초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 1280-4,서울특별시 성북구 길음로15길 55,37.609065,127.016798,POINT (127.01680 37.60907)
550,서울길음초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 635-7,서울특별시 성북구 길음로 142,37.613631,127.016659,POINT (127.01666 37.61363)
551,서울미아초등학교,성북구,1129066000,길음제1동,서울특별시 성북구 길음동 1168,서울특별시 성북구 삼양로 77,37.610589,127.021646,POINT (127.02165 37.61059)


In [162]:
# Train 데이터 좌표 위치 
center = (37.467548,	126.890947)
zoom = 12

m = folium.Map(location=center, zoom_start=zoom, tiles='cartodbpositron')
folium.Choropleth(
    geo_data = target_geo,
    data = target_ele_pop,
    columns = ('행정동', '초등학생_인구'),
    key_on = 'feature.properties.읍면동명',
    fill_color = 'Blues',
    legend_name = '초등학생_인구'
).add_to(m)

# 행정동 naming 
for index, row in target_geo.iterrows():
    title = f"{row['시군구명']} - {row['읍면동명']}"
    folium.CircleMarker([row['위도'], row['경도']], popup=title, radius =1,color = 'white').add_to(m)

# 초등학교 위치
for index, row in df_academy.iterrows():
    title = f"{row['학원명']}"
    # folium.Marker([row['위도'], row['경도']],popup=title, icon=folium.Icon(icon ='dumbell', color = 'orange')).add_to(m)  
    folium.Circle([row['위도'], row['경도']], radius =0.5, fill='#ffa5a5', color = '#ffa5a5').add_to(m)

# 초등학교 위치
for index, row in target_school.iterrows():
    title = f"{row['학교명']}"
    folium.Marker([row['위도'], row['경도']],popup=title, icon=folium.Icon(icon ='dumbell', color = 'orange')).add_to(m)  
    folium.Circle([row['위도'], row['경도']], radius =881, fill='blue', color = 'blue').add_to(m)

# # 전체 target 좌표
# for index, row in pop_30.iterrows():  
#     folium.Circle([row['위도'], row['경도']],
#                   radius = 3, color = 'red' ).add_to(m)   
# 행정동 내 좌표 
# for index, row in df_pop.iterrows():  
#     folium.Circle([row['위도'], row['경도']],
#                   radius = 3, color = 'blue' ).add_to(m)
               
m

Output hidden; open in https://colab.research.google.com to view.

## 돌봄시설 

In [106]:
merge_base = df_pop[['gid', '행정동코드', '행정동','위도', '경도']]
merge_base

,gid,행정동코드,행정동,위도,경도
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929
...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587


### 우리마을키움센터

In [107]:
# 전처리 후
df_kium = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/키움센터_위치_행정동.geojson')
df_kium

,자치구,행정동코드,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [108]:
df_kium[df_kium['시설ID']=='구로7호점']

,자치구,행정동코드,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
75,구로구,1153054000,구로제3동,구로7호점,구로7호점 융합형 우리동네키움센터,30,6,서울특별시 구로구 디지털로32가길 50,37.481338,126.89774,POINT (126.89774 37.48134)


In [109]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(df_point, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(df_point['gid'],df_point['위도'],df_point['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [110]:
lscp_kium=pd.DataFrame(harv_dis(merge_base,df_kium,881), columns=['gid','키움센터','키움센터_거리'])
lscp_kium

,gid,키움센터,키움센터_거리
0,다사46aa48aa,영등포2호점,638.776183
1,다사46aa48ab,영등포2호점,863.860886
2,다사59aa58bb,노원13호점,838.591704
3,다사59aa59aa,노원13호점,853.096476
4,다사59ab59aa,노원13호점,610.984760
5,다사59ab59ab,노원13호점,723.466655


In [111]:
lscp_kium = lscp_kium.merge(df_kium, left_on='키움센터', right_on='시설ID')[['gid', '키움센터', '키움센터_거리', '상시돌봄', '일시돌봄']]
lscp_kium

,gid,키움센터,키움센터_거리,상시돌봄,일시돌봄
0,다사46aa48aa,영등포2호점,638.776183,25,5
1,다사46aa48ab,영등포2호점,863.860886,25,5
2,다사59aa58bb,노원13호점,838.591704,28,4
3,다사59aa59aa,노원13호점,853.096476,28,4
4,다사59ab59aa,노원13호점,610.984760,28,4
5,다사59ab59ab,노원13호점,723.466655,28,4


In [112]:
#격자별로 groupby 
df_loc = pd.DataFrame(lscp_kium, columns = ['gid','키움센터','키움센터_거리']) 
lscp_kium_groupby = dict(list(df_loc.groupby('gid')))

# 전체 격자 리스트를 데이터 프레임으로 만들기 
df_kium_loc = pd.DataFrame(lscp_kium_groupby.items())
df_kium_loc['gid'] = np.array(np.nan)

# 각 격자의 가까운 키움센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_kium_groupby.keys()):
  stations.append(lscp_kium_groupby[k]['키움센터'].values)

#격자와 각 격자별 키움센터 데이터 프레임 합치기 
df_kium_loc['키움센터'] = stations
df_kium_loc = df_kium_loc[[0, '키움센터']]
df_kium_loc.columns = ['gid', '키움센터']
df_kium_loc

,gid,키움센터
0,다사46aa48aa,[영등포2호점]
1,다사46aa48ab,[영등포2호점]
2,다사59aa58bb,[노원13호점]
3,다사59aa59aa,[노원13호점]
4,다사59ab59aa,[노원13호점]
5,다사59ab59ab,[노원13호점]


In [113]:
df_kium_loc['키움센터_개수'] = df_kium_loc['키움센터'].apply(lambda x: len(x))
df_kium_loc['키움센터_상시돌봄정원'] = list(lscp_kium.groupby(['gid']).agg('sum')['상시돌봄'])
df_kium_loc['키움센터_일시돌봄정원'] = list(lscp_kium.groupby(['gid']).agg('sum')['일시돌봄'])
df_kium_loc

,gid,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원
0,다사46aa48aa,[영등포2호점],1,25,5
1,다사46aa48ab,[영등포2호점],1,25,5
2,다사59aa58bb,[노원13호점],1,28,4
3,다사59aa59aa,[노원13호점],1,28,4
4,다사59ab59aa,[노원13호점],1,28,4
5,다사59ab59ab,[노원13호점],1,28,4


### 초등돌봄센터

In [114]:
df_ele_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson')
df_ele_dolbom

,행정동코드,구,행정동,학교명,소재지도로명주소,초등돌봄교실_교실개수,초등돌봄교실_이용인원,초등학교_위도,초등학교_경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153078000,구로구,오류제2동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824)


In [115]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(df_point, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('학교명')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(df_point['gid'],df_point['위도'],df_point['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['초등학교_위도'],target['초등학교_경도']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [116]:
lscp_ele_dolbom=pd.DataFrame(harv_dis(merge_base,df_ele_dolbom,881), columns=['gid','초등돌봄교실','초등돌봄교실_거리'])
lscp_ele_dolbom

,gid,초등돌봄교실,초등돌봄교실_거리
0,다사57ab56ab,서울숭덕초등학교,400.793206
1,다사57ab56ab,서울길원초등학교,362.722421
2,다사57ab56ab,서울길음초등학교,870.397263
3,다사57ab56ab,서울미아초등학교,666.253125
4,다사57ab56ab,서울정수초등학교,855.296027
...,...,...,...
229,다사59ab59aa,서울번동초등학교,191.847863
230,다사59ab59ab,서울신창초등학교,462.194531
231,다사59ab59ab,서울신화초등학교,118.862521
232,다사59ab59ab,서울번동초등학교,440.142754


In [117]:
lscp_ele_dolbom = lscp_ele_dolbom.merge(df_ele_dolbom, left_on='초등돌봄교실', right_on='학교명')[['gid', '초등돌봄교실', '초등돌봄교실_거리','초등돌봄교실_교실개수', '초등돌봄교실_이용인원']]
lscp_ele_dolbom

,gid,초등돌봄교실,초등돌봄교실_거리,초등돌봄교실_교실개수,초등돌봄교실_이용인원
0,다사57ab56ab,서울숭덕초등학교,400.793206,4.0,83.0
1,다사57ab56ba,서울숭덕초등학교,577.568595,4.0,83.0
2,다사57ab56bb,서울숭덕초등학교,795.043945,4.0,83.0
3,다사57ba56aa,서울숭덕초등학교,585.422481,4.0,83.0
4,다사57ba56ab,서울숭덕초등학교,624.592601,4.0,83.0
...,...,...,...,...,...
229,다사59aa59aa,서울신창초등학교,807.965691,3.0,74.0
230,다사59aa59ab,서울신창초등학교,622.205690,3.0,74.0
231,다사59ab59aa,서울신창초등학교,692.305767,3.0,74.0
232,다사59ab59ab,서울신창초등학교,462.194531,3.0,74.0


In [118]:
#격자별로 groupby 
df_loc = pd.DataFrame(lscp_ele_dolbom, columns = ['gid','초등돌봄교실','초등돌봄교실_거리']) 
lscp_ele_dolbom_groupby = dict(list(df_loc.groupby('gid')))

# 전체 격자 리스트를 데이터 프레임으로 만들기 
df_ele_dolbom_loc = pd.DataFrame(lscp_ele_dolbom_groupby.items())
df_ele_dolbom_loc['gid'] = np.array(np.nan)

# 각 격자의 가까운 초등돌봄교실 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_ele_dolbom_groupby.keys()):
  stations.append(lscp_ele_dolbom_groupby[k]['초등돌봄교실'].values)

#격자와 각 격자별 초등돌봄교실 데이터 프레임 합치기 
df_ele_dolbom_loc['초등돌봄교실'] = stations
df_ele_dolbom_loc = df_ele_dolbom_loc[[0, '초등돌봄교실']]
df_ele_dolbom_loc.columns = ['gid', '초등돌봄교실']
df_ele_dolbom_loc

,gid,초등돌봄교실
0,다사44aa48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
1,다사44ab48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
2,다사44ab48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
3,다사44ab48ba,"[서울정목초등학교, 서울영도초등학교, 서울월촌초등학교]"
4,다사44ba48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]"
...,...,...
57,다사59aa58bb,"[서울번동초등학교, 서울화계초등학교, 서울신화초등학교]"
58,다사59aa59aa,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]"
59,다사59aa59ab,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]"
60,다사59ab59aa,"[서울번동초등학교, 서울신화초등학교, 서울신창초등학교]"


In [119]:
df_ele_dolbom_loc['초등돌봄교실_개수'] = df_ele_dolbom_loc['초등돌봄교실'].apply(lambda x: len(x))
df_ele_dolbom_loc['초등돌봄교실_교실개수'] = list(lscp_ele_dolbom.groupby(['gid']).agg('sum')['초등돌봄교실_교실개수'])
df_ele_dolbom_loc['초등돌봄교실_이용인원'] = list(lscp_ele_dolbom.groupby(['gid']).agg('sum')['초등돌봄교실_이용인원'])
df_ele_dolbom_loc

,gid,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원
0,다사44aa48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",4,14.0,295.0
1,다사44ab48aa,"[서울서정초등학교, 서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",4,14.0,295.0
2,다사44ab48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",3,10.0,216.0
3,다사44ab48ba,"[서울정목초등학교, 서울영도초등학교, 서울월촌초등학교]",3,10.0,229.0
4,다사44ba48ab,"[서울목운초등학교, 서울정목초등학교, 서울영도초등학교]",3,10.0,216.0
...,...,...,...,...,...
57,다사59aa58bb,"[서울번동초등학교, 서울화계초등학교, 서울신화초등학교]",3,13.0,236.0
58,다사59aa59aa,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]",4,18.0,334.0
59,다사59aa59ab,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]",4,18.0,334.0
60,다사59ab59aa,"[서울번동초등학교, 서울신화초등학교, 서울신창초등학교]",3,12.0,213.0


### 지역아동센터


In [120]:
# 전처리 후
df_local =  gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/지역아동센터_위치포함.geojson')
df_local

,행정동코드,시군구,행정동,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry
0,1111067000,종로구,창신제1동,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312)
1,1111068000,종로구,창신제2동,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730)
2,1111064000,종로구,이화동,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776)
3,1111068000,종로구,창신제2동,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509)
4,1111070000,종로구,숭인제1동,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016529,POINT (127.01653 37.57785)
...,...,...,...,...,...,...,...,...,...
451,1174065000,강동구,성내제2동,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537)
452,1174057000,강동구,암사제1동,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055)
453,1174066000,강동구,성내제3동,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823)
454,1174052000,강동구,상일동,구립함께하는지역아동센터,19,서울 강동구 구천면로 656,37.550608,127.170775,POINT (127.17077 37.55061)


In [121]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(df_point, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(df_point['gid'],df_point['위도'],df_point['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [122]:
lscp_local=pd.DataFrame(harv_dis(merge_base,df_local,881), columns=['gid','지역아동센터','지역아동센터_거리'])
lscp_local

,gid,지역아동센터,지역아동센터_거리
0,다사57ab56ab,꿈나래지역아동센터,390.052441
1,다사57ab56ab,돈암동교회모퉁이돌지역아동센터,876.241680
2,다사57ab56ab,구립길음동꿈나무키우미돌봄센터,206.808344
3,다사57ab56ba,시습지역아동센터,706.921943
4,다사57ab56ba,성광지역아동센터,873.196099
...,...,...,...
147,다사59ab59aa,구립초안산숲속지역아동센터,450.765464
148,다사59ab59ab,독립문지역아동센터,848.830478
149,다사59ab59ab,아름다운땅지역아동센터,457.368875
150,다사59ab59ab,창동드림지역아동센터,273.462246


In [123]:
lscp_local = lscp_local.merge(df_local, left_on='지역아동센터', right_on='시설명')[['gid', '지역아동센터', '지역아동센터_거리', '입소정원(명)']]
lscp_local

,gid,지역아동센터,지역아동센터_거리,입소정원(명)
0,다사57ab56ab,꿈나래지역아동센터,390.052441,26
1,다사57ab56ab,꿈나래지역아동센터,390.052441,39
2,다사57ab56ab,꿈나래지역아동센터,390.052441,19
3,다사57ab56ba,꿈나래지역아동센터,607.376141,26
4,다사57ab56ba,꿈나래지역아동센터,607.376141,39
...,...,...,...,...
188,다사59aa58bb,구립초안산숲속지역아동센터,782.340543,19
189,다사59aa59aa,구립초안산숲속지역아동센터,692.331552,19
190,다사59aa59ab,구립초안산숲속지역아동센터,687.130315,19
191,다사59ab59aa,구립초안산숲속지역아동센터,450.765464,19


In [124]:
#격자별로 groupby 
df_loc = pd.DataFrame(lscp_local, columns = ['gid','지역아동센터','지역아동센터_거리']) 
lscp_local_groupby = dict(list(df_loc.groupby('gid')))

# 전체 격자 리스트를 데이터 프레임으로 만들기 
df_local_loc = pd.DataFrame(lscp_local_groupby.items())
df_local_loc['gid'] = np.array(np.nan)

# 각 격자의 가까운 지역아동센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_local_groupby.keys()):
  stations.append(lscp_local_groupby[k]['지역아동센터'].values)

#격자와 각 격자별 지역아동센터 데이터 프레임 합치기 
df_local_loc['지역아동센터'] = stations
df_local_loc = df_local_loc[[0, '지역아동센터']]
df_local_loc.columns = ['gid', '지역아동센터']
df_local_loc

,gid,지역아동센터
0,다사44aa48aa,[화곡지역아동센터]
1,다사44ab48ab,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]"
2,다사44ab48ba,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]"
3,다사44ba48ab,[기린청소년지역아동센터]
4,다사44ba48ba,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]"
5,다사44ba48bb,"[그리스도의교회지역아동센터, 기린청소년지역아동센터, 나무와숲지역아동센터]"
6,다사44bb48ba,[기린청소년지역아동센터]
7,다사44bb48bb,"[기린청소년지역아동센터, 나무와숲지역아동센터]"
8,다사44bb49aa,"[기린청소년지역아동센터, 나무와숲지역아동센터, 샘물지역아동센터]"
9,다사45aa48bb,[기린청소년지역아동센터]


In [125]:
df_local_loc['지역아동센터_개수'] = df_local_loc['지역아동센터'].apply(lambda x: len(x))
df_local_loc['지역아동센터_정원'] = list(lscp_local.groupby(['gid']).agg('sum')['입소정원(명)'])
df_local_loc

,gid,지역아동센터,지역아동센터_개수,지역아동센터_정원
0,다사44aa48aa,[화곡지역아동센터],1,39
1,다사44ab48ab,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]",2,54
2,다사44ab48ba,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]",2,54
3,다사44ba48ab,[기린청소년지역아동센터],1,19
4,다사44ba48ba,"[그리스도의교회지역아동센터, 기린청소년지역아동센터]",2,54
5,다사44ba48bb,"[그리스도의교회지역아동센터, 기린청소년지역아동센터, 나무와숲지역아동센터]",3,89
6,다사44bb48ba,[기린청소년지역아동센터],1,19
7,다사44bb48bb,"[기린청소년지역아동센터, 나무와숲지역아동센터]",2,54
8,다사44bb49aa,"[기린청소년지역아동센터, 나무와숲지역아동센터, 샘물지역아동센터]",3,77
9,다사45aa48bb,[기린청소년지역아동센터],1,19


In [126]:
merge_base.merge(df_local_loc,on='gid', how='left')

,gid,행정동코드,행정동,위도,경도,지역아동센터,지역아동센터_개수,지역아동센터_정원
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 돈암동교회모퉁이돌지역아...",5.0,152.0
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 구립길음동꿈나무키우미돌...",6.0,176.0
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 구립길음동꿈나무키우미돌...",6.0,176.0
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038,"[구립길음동꿈나무키우미돌봄센터, 시습지역아동센터, 성광지역아동센터, 돌산공부방지역아...",6.0,159.0
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 돈암동교회모퉁이돌지역아...",7.0,204.0
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768,"[아름다운땅지역아동센터, 창동드림지역아동센터, 삼동지역아동센터, 지역아동센터사랑의공...",5.0,114.0
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754,"[독립문지역아동센터, 아름다운땅지역아동센터, 창동드림지역아동센터, 구립초안산숲속지역...",4.0,95.0
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740,"[독립문지역아동센터, 아름다운땅지역아동센터, 강북구공립지역아동센터, 창동드림지역아동...",5.0,127.0
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587,"[아름다운땅지역아동센터, 창동드림지역아동센터, 삼동지역아동센터, 지역아동센터사랑의공...",5.0,114.0


### 방과후아카데미


In [127]:
#전처리 후 
df_after =  gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/방과후아카데미_위치포함.geojson')
df_after

,행정동코드,시군구,행정동,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry
0,1114065000,중구,신당제5동,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022778,POINT (127.02278 37.56505)
1,1114063500,중구,약수동,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
2,1120056000,성동구,행당제1동,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306)
3,1123070500,동대문구,청량리동,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727)
4,1126065500,중랑구,망우본동,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588)
5,1126054000,중랑구,면목제4동,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317)
6,1129078000,성북구,장위제3동,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757)
7,1130564500,강북구,우이동,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594)
8,1132051400,도봉구,창제4동,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827)
9,1138059000,은평구,응암제2동,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415)


In [128]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(df_point, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(df_point['gid'],df_point['위도'],df_point['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [129]:
lscp_after=pd.DataFrame(harv_dis(merge_base,df_after,881), columns=['gid','방과후아카데미','방과후아카데미_거리'])
lscp_after

,gid,방과후아카데미,방과후아카데미_거리
0,다사44aa48aa,시립화곡청소년센터,772.959005
1,다사44ab48ba,시립화곡청소년센터,875.857606


In [130]:
lscp_after = lscp_after.merge(df_after, left_on='방과후아카데미', right_on='시설명')[['gid', '방과후아카데미', '방과후아카데미_거리', '초등학생 수용인원(정원)']]
lscp_after

,gid,방과후아카데미,방과후아카데미_거리,초등학생 수용인원(정원)
0,다사44aa48aa,시립화곡청소년센터,772.959005,20
1,다사44ab48ba,시립화곡청소년센터,875.857606,20


In [131]:
#격자별로 groupby 
df_loc = pd.DataFrame(lscp_after, columns = ['gid','방과후아카데미','방과후아카데미_거리']) 
lscp_after_groupby = dict(list(df_loc.groupby('gid')))

# 전체 격자 리스트를 데이터 프레임으로 만들기 
df_after_loc = pd.DataFrame(lscp_after_groupby.items())
df_after_loc['gid'] = np.array(np.nan)

# 각 격자의 가까운 방과후아카데미 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_after_groupby.keys()):
  stations.append(lscp_after_groupby[k]['방과후아카데미'].values)

#격자와 각 격자별 방과후아카데미 데이터 프레임 합치기 
df_after_loc['방과후아카데미'] = stations
df_after_loc = df_after_loc[[0, '방과후아카데미']]
df_after_loc.columns = ['gid', '방과후아카데미']
df_after_loc

,gid,방과후아카데미
0,다사44aa48aa,[시립화곡청소년센터]
1,다사44ab48ba,[시립화곡청소년센터]


In [132]:
df_after_loc

,gid,방과후아카데미
0,다사44aa48aa,[시립화곡청소년센터]
1,다사44ab48ba,[시립화곡청소년센터]


In [133]:
# 방과후 아카데미 없음 0으로 처리 
df_after_loc['방과후아카데미_개수'] = df_after_loc['방과후아카데미'].apply(lambda x: len(x))
df_after_loc['방과후아카데미_정원'] = list(lscp_after.groupby(['gid']).agg('sum')['초등학생 수용인원(정원)'])

# df_after_loc['방과후아카데미_개수'] = 0
# df_after_loc['방과후아카데미_정원'] = 0
# df_after_loc

In [134]:
merge_base.merge(df_after_loc,on='gid', how='left')

,gid,행정동코드,행정동,위도,경도,방과후아카데미,방과후아카데미_개수,방과후아카데미_정원
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082,NaN,NaN,NaN
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067,NaN,NaN,NaN
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053,NaN,NaN,NaN
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038,NaN,NaN,NaN
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768,NaN,NaN,NaN
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754,NaN,NaN,NaN
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740,NaN,NaN,NaN
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587,NaN,NaN,NaN


### merge

In [135]:
merge_base = df_pop[['gid', '행정동코드', '행정동','위도', '경도']]
merge_base

,gid,행정동코드,행정동,위도,경도
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929
...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587


In [136]:
df1 = merge_base.merge(df_kium_loc,on='gid', how='left')
df2 = df1.merge(df_ele_dolbom_loc, on = 'gid', how = 'left')
df3 = df2.merge(df_local_loc, on='gid', how = 'left')
df4 = df3.merge(df_after_loc, on ='gid', how = 'left')

df4

,gid,행정동코드,행정동,위도,경도,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미,방과후아카데미_개수,방과후아카데미_정원
0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082,NaN,NaN,NaN,NaN,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울정수초...",5,19.0,415.0,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 돈암동교회모퉁이돌지역아...",5.0,152.0,NaN,NaN,NaN
1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067,NaN,NaN,NaN,NaN,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교]",4,17.0,373.0,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 구립길음동꿈나무키우미돌...",6.0,176.0,NaN,NaN,NaN
2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053,NaN,NaN,NaN,NaN,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산...",6,24.0,529.0,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 구립길음동꿈나무키우미돌...",6.0,176.0,NaN,NaN,NaN
3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038,NaN,NaN,NaN,NaN,"[서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산초등학교, 서울송천...",5,20.0,446.0,"[구립길음동꿈나무키우미돌봄센터, 시습지역아동센터, 성광지역아동센터, 돌산공부방지역아...",6.0,159.0,NaN,NaN,NaN
4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929,NaN,NaN,NaN,NaN,"[서울숭덕초등학교, 서울길원초등학교, 서울미아초등학교, 서울정덕초등학교, 서울개운초...",6,21.0,442.0,"[꿈나래지역아동센터, 꿈나래지역아동센터, 꿈나래지역아동센터, 돈암동교회모퉁이돌지역아...",7.0,204.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,번2동,37.628423,127.036768,[노원13호점],1.0,28.0,4.0,"[서울번동초등학교, 서울화계초등학교, 서울신화초등학교]",3,13.0,236.0,"[아름다운땅지역아동센터, 창동드림지역아동센터, 삼동지역아동센터, 지역아동센터사랑의공...",5.0,114.0,NaN,NaN,NaN
58,다사59aa59aa,1130560300,번2동,37.630677,127.036754,[노원13호점],1.0,28.0,4.0,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]",4,18.0,334.0,"[독립문지역아동센터, 아름다운땅지역아동센터, 창동드림지역아동센터, 구립초안산숲속지역...",4.0,95.0,NaN,NaN,NaN
59,다사59aa59ab,1130560300,번2동,37.632930,127.036740,NaN,NaN,NaN,NaN,"[서울번동초등학교, 서울수송초등학교, 서울신화초등학교, 서울신창초등학교]",4,18.0,334.0,"[독립문지역아동센터, 아름다운땅지역아동센터, 강북구공립지역아동센터, 창동드림지역아동...",5.0,127.0,NaN,NaN,NaN
60,다사59ab59aa,1130560300,번2동,37.630688,127.039587,[노원13호점],1.0,28.0,4.0,"[서울번동초등학교, 서울신화초등학교, 서울신창초등학교]",3,12.0,213.0,"[아름다운땅지역아동센터, 창동드림지역아동센터, 삼동지역아동센터, 지역아동센터사랑의공...",5.0,114.0,NaN,NaN,NaN


In [137]:
df4.to_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_돌봄시설정보.pickle')

In [144]:
df4.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_돌봄시설정보.csv')

In [138]:
df = pd.read_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_인구.pickle')
df

,gid,행정동코드,시군구명,행정동,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,경도,위도,geometry
0,다사57ab56ab,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울정수초...",5,153.0,422.0,530.0,2074.0,127.017082,37.605811,POINT (127.01708 37.60581)
1,다사57ab56ba,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교]",4,430.0,570.0,948.0,2876.0,127.017067,37.608064,POINT (127.01707 37.60806)
2,다사57ab56bb,1129066000,성북구,길음제1동,"[서울숭덕초등학교, 서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산...",6,349.0,350.0,737.0,2205.0,127.017053,37.610318,POINT (127.01705 37.61032)
3,다사57ab57aa,1129066000,성북구,길음제1동,"[서울길원초등학교, 서울길음초등학교, 서울미아초등학교, 서울삼각산초등학교, 서울송천...",5,330.0,255.0,756.0,2353.0,127.017038,37.612571,POINT (127.01704 37.61257)
4,다사57ba56aa,1129066000,성북구,길음제1동,"[서울정덕초등학교, 서울숭덕초등학교, 서울길원초등학교, 서울미아초등학교, 서울개운초...",6,277.0,472.0,783.0,2759.0,127.019929,37.603569,POINT (127.01993 37.60357)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,다사59aa58bb,1130560300,강북구,번2동,"[서울신화초등학교, 서울번동초등학교, 서울화계초등학교]",3,49.0,182.0,238.0,1000.0,127.036768,37.628423,POINT (127.03677 37.62842)
58,다사59aa59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,79.0,182.0,215.0,1093.0,127.036754,37.630677,POINT (127.03675 37.63068)
59,다사59aa59ab,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교, 서울수송초등학교]",4,22.0,86.0,191.0,822.0,127.036740,37.632930,POINT (127.03674 37.63293)
60,다사59ab59aa,1130560300,강북구,번2동,"[서울신창초등학교, 서울신화초등학교, 서울번동초등학교]",3,86.0,256.0,338.0,1407.0,127.039587,37.630688,POINT (127.03959 37.63069)


In [139]:
df.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_인구.csv', index = False)

In [151]:
pd.read_csv('/content/drive/My Drive/Project/Seoul/가공데이터/동3개/1109_동3개_초등학교881m_학원정보.csv')

,Unnamed: 0,gid,행정동코드,행정동,위도,경도,학원명,학원개수
0,0,다사57ab56ab,1129066000,길음제1동,37.605811,127.017082,['으뜸보습학원' '아이즈학원' '투탑학원' '남림학원' '스타트보습학원' '소마성...,62
1,1,다사57ab56ba,1129066000,길음제1동,37.608064,127.017067,['으뜸보습학원' '아이즈학원' '남림학원' '스타트보습학원' '소마성북학원' '더...,54
2,2,다사57ab56bb,1129066000,길음제1동,37.610318,127.017053,['으뜸보습학원' '아이즈학원' '스타트보습학원' '소마성북학원' '더웨이(theW...,55
3,3,다사57ab57aa,1129066000,길음제1동,37.612571,127.017038,['아이즈학원' '가온수학학원' '스타트보습학원' '소마성북학원' '참조은학원' '...,58
4,4,다사57ba56aa,1129066000,길음제1동,37.603569,127.019929,['으뜸보습학원' '아이즈학원' '투탑학원' '남림학원' '스타트보습학원' '소마성...,68
...,...,...,...,...,...,...,...,...
57,57,다사59aa58bb,1130560300,번2동,37.628423,127.036768,['캠브리지영어학원' '최윤영어학원' '최윤2관학원' '엠베스트에스이(SE)&뮤엠영...,14
58,58,다사59aa59aa,1130560300,번2동,37.630677,127.036754,['대성탑(TOP)학원' '캠브리지영어학원' '최윤영어학원' '최윤2관학원' '엠베...,20
59,59,다사59aa59ab,1130560300,번2동,37.632930,127.036740,['S.E.명문학원' '대성탑(TOP)학원' '박정화뮤엠영어학원' '캠브리지영어학원...,24
60,60,다사59ab59aa,1130560300,번2동,37.630688,127.039587,['대성탑(TOP)학원' '캠브리지영어학원' '최윤영어학원' '최윤2관학원' '엠베...,15
